<a href="https://www.kaggle.com/code/dzulfikrialwi/eda-young-player?scriptVersionId=248397818" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Import & Setup

In [17]:
# Mengimpor library yang diperlukan untuk analisis data dan visualisasi.
# 'pandas' untuk manipulasi data, 'seaborn' untuk pengaturan gaya plot.
# 'warnings' untuk mengabaikan pesan peringatan yang tidak penting.
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# Mengatur gaya visualisasi plot menggunakan seaborn.
plt.style.use("seaborn")
sns.set(font_scale=1.1)

# Mencetak daftar file yang ada di direktori input Kaggle.
# Ini membantu memastikan data yang diperlukan tersedia.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/all-football-players-stats-in-top-5-leagues-2425/top5-players24-25.xlsx


## 💡 Latar Belakang

Dalam industri sepakbola modern, pemain muda bukan sekadar pelapis skuad — mereka adalah investasi jangka panjang klub. Jadwal pertandingan yang semakin padat, tekanan komersial, dan risiko cedera pada pemain inti membuat kebutuhan akan regenerasi semakin mendesak.

Di sisi lain, klub kini memiliki akses pada data performa yang sangat detail. Hal ini membuka peluang untuk mengevaluasi kontribusi pemain muda secara objektif — tidak hanya berdasarkan gol dan assist, tetapi juga melalui metrik lanjutan seperti expected goals (xG), expected assists (xAG), serta aksi progresif lainnya.

Analisis ini bertujuan untuk melihat bagaimana performa pemain muda di liga top Eropa. Sejauh mana mereka memberikan kontribusi? Apakah mereka siap menjadi andalan atau masih butuh waktu berkembang? Temuan ini diharapkan dapat memberi gambaran bagi klub dalam merancang strategi rotasi dan investasi masa depan.



## 🎯 Tujuan Analisis
 Notebook ini bertujuan untuk mengeksplorasi performa pemain muda (usia <= 21 tahun) dari 5 liga top Eropa (Premier League, La Liga, Bundesliga, Serie A, Ligue 1). Fokus utama analisis mencakup:
 - Frekuensi dan pola menit bermain pemain muda
 - Produktivitas dalam mencetak gol dan membuat assist
 - Efektivitas pemain dalam konversi xG (Expected Goals) dan xAG (Expected Assisted Goals)
 - Distribusi posisi bermain pemain muda
 - Identifikasi liga dan klub yang paling mendukung pengembangan talenta muda


## 🔍 Metodologi
- Dataset: Statistik pemain dari lima liga top Eropa musim terbaru.

- Pemain Muda: Didefinisikan sebagai pemain dengan usia kurang atau sama dengan umur 21 tahun (<=21).

- Statistik Kunci:

    - Finishing: Goals, Non-Penalty Goals, xG, Gls/90, Gls/xG

    - Playmaking: Assists, xAG, Progressive Passes, Ast/90

    - Defending: Minutes, Position, Card, and non-goal contributions

- Tools:

    - Python (Pandas, Plotly, Seaborn, Scikit-learn)

- Clustering: Pemain dikelompokkan berdasarkan peran dominan (Finisher, Playmaker, Defender) melalui pendekatan KMeans & radar profiling.

# 2. Load Data

In [18]:
# Memuat data dari file Excel ke dalam DataFrame pandas.
# Data ini berisi statistik pemain dari 5 liga top Eropa musim 2024-2025.
PATH = '/kaggle/input/all-football-players-stats-in-top-5-leagues-2425/top5-players24-25.xlsx'
df = pd.read_excel(PATH)
# Menampilkan 5 baris pertama dari DataFrame untuk melihat sekilas struktur data.
df.head(5)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Gls_90,Ast_90,G+A_90,G-PK_90,G+A-PK_90,xG_90,xAG_90,xG+xAG_90,npxG_90,npxG+xAG_90
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24.0,2000.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1,2,Max Aarons,eng ENG,"DF,MF",Valencia,es La Liga,24.0,2000.0,4,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.02,0.02,0.00,0.02
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21.0,2003.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.10,0.00,0.10,0.10,0.10
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,20.0,2004.0,6,0,...,0.0,0.0,0.0,0.0,0.0,0.06,0.00,0.06,0.06,0.06
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18.0,2006.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00


## 📊 Pemeriksaan dan Pembersihan Data Awal
Bagian ini bertujuan untuk memeriksa kualitas data, seperti keberadaan nilai yang hilang (missing values) dan duplikasi data, serta melakukan pembersihan dasar jika diperlukan.

In [19]:
#Check missing values
print("\n--- Total Missing Values on All Dataset ---")
total_missing = df.isnull().sum().sum()
if total_missing > 0:
    print(f"Total nilai hilang: {total_missing}")
else:
    print("No missing value on dataset.")

print("\n--- Missing Values per columns (if any) ---")
missing_per_column = df.isnull().sum()
missing_per_column = missing_per_column[missing_per_column > 0]
if not missing_per_column.empty:
    print(missing_per_column)
else:
    print("No missing value per columns.")


--- Total Missing Values on All Dataset ---
Total nilai hilang: 23

--- Missing Values per columns (if any) ---
Nation    7
Age       8
Born      8
dtype: int64


In [20]:
# Memeriksa total nilai yang hilang di seluruh dataset.
print("\n--- Total Nilai Hilang di Seluruh Dataset ---")
total_missing = df.isnull().sum().sum()
if total_missing > 0:
    print(f"Total nilai hilang: {total_missing}")
else:
    print("Tidak ada nilai hilang di dataset.")

# Memeriksa nilai yang hilang per kolom (jika ada).
print("\n--- Nilai Hilang per Kolom (jika ada) ---")
missing_per_column = df.isnull().sum()
missing_per_column = missing_per_column[missing_per_column > 0]
if not missing_per_column.empty:
    print(missing_per_column)
else:
    print("Tidak ada nilai hilang per kolom.")

# Memeriksa total nilai duplikat di seluruh dataset.
print("\n--- Total Nilai Duplikat di Seluruh Dataset ---")
total_duplicate = df.duplicated().sum().sum()
if total_duplicate > 0:
    print(f"Total nilai duplikat: {total_duplicate}")
else:
    print("Tidak ada nilai duplikat di dataset.")

# Menampilkan baris-baris yang memiliki nilai hilang pada kolom 'Nation', 'Age', atau 'Born'.
missing_rows = df[df['Nation'].isnull() | df['Age'].isnull() | df['Born'].isnull()][['Player', 'Squad','Nation', 'Age', 'Born']]
print(missing_rows)

# Menghapus baris yang mengandung nilai hilang untuk memastikan data bersih.
df_cleaned = df.dropna()

# Memverifikasi kembali nilai yang hilang setelah pembersihan.
print("\n--- Nilai Hilang per Kolom (setelah pembersihan) ---")
missing_per_column = df_cleaned.isnull().sum()
missing_per_column = missing_per_column[missing_per_column > 0]
if not missing_per_column.empty:
    print(missing_per_column)
else:
    print("Tidak ada nilai hilang per kolom.")



--- Total Nilai Hilang di Seluruh Dataset ---
Total nilai hilang: 23

--- Nilai Hilang per Kolom (jika ada) ---
Nation    7
Age       8
Born      8
dtype: int64

--- Total Nilai Duplikat di Seluruh Dataset ---
Tidak ada nilai duplikat di dataset.
                   Player           Squad  Nation   Age    Born
100         Olabade Aluko  Leicester City     NaN   NaN     NaN
273        Hannes Behrens      Hoffenheim  de GER   NaN     NaN
663   Pape Daouda Diongue      Strasbourg  sn SEN   NaN     NaN
862            Jake Evans  Leicester City     NaN   NaN     NaN
1320       Atakan Karazor       Stuttgart     NaN  27.0  1996.0
1518            Fer López      Celta Vigo     NaN   NaN     NaN
1603          Mateus Mane          Wolves     NaN   NaN     NaN
1769        Max Moerstedt      Hoffenheim  de GER   NaN     NaN
1778         Jeremy Monga  Leicester City     NaN   NaN     NaN
1917       Plamedi Nsingi          Nantes     NaN  23.0  2000.0

--- Nilai Hilang per Kolom (setelah pembersihan

In [21]:
# Menampilkan 5 baris pertama dari DataFrame yang sudah bersih.
df_cleaned.head()

# Menampilkan informasi ringkas tentang DataFrame (tipe data, jumlah non-null).
df_cleaned.info()

# Menampilkan statistik deskriptif untuk kolom numerik (rata-rata, standar deviasi, dll.).
df_cleaned.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 2844 entries, 0 to 2853
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           2844 non-null   int64  
 1   Player       2844 non-null   object 
 2   Nation       2844 non-null   object 
 3   Pos          2844 non-null   object 
 4   Squad        2844 non-null   object 
 5   Comp         2844 non-null   object 
 6   Age          2844 non-null   float64
 7   Born         2844 non-null   float64
 8   MP           2844 non-null   int64  
 9   Starts       2844 non-null   int64  
 10  Min          2844 non-null   int64  
 11  90s          2844 non-null   float64
 12  Gls          2844 non-null   int64  
 13  Ast          2844 non-null   int64  
 14  G+A          2844 non-null   int64  
 15  G-PK         2844 non-null   int64  
 16  PK           2844 non-null   int64  
 17  PKatt        2844 non-null   int64  
 18  CrdY         2844 non-null   int64  
 19  CrdR       

,Rk,Age,Born,MP,Starts,Min,90s,Gls,Ast,G+A,...,Gls_90,Ast_90,G+A_90,G-PK_90,G+A-PK_90,xG_90,xAG_90,xG+xAG_90,npxG_90,npxG+xAG_90
count,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,...,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000,2844.000000
mean,1428.365682,25.018987,1998.637482,19.061181,13.537271,1215.077707,13.500141,1.688115,1.204290,2.892405,...,0.112032,0.076909,0.188949,0.104434,0.181357,0.135591,0.088878,0.224272,0.127690,0.216428
std,824.506208,4.492890,4.499159,11.499917,11.324974,965.234586,10.725007,3.160629,1.948538,4.538927,...,0.196378,0.136747,0.264578,0.186588,0.253648,0.218226,0.124636,0.272760,0.209537,0.263716
min,1.000000,15.000000,1982.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,714.750000,22.000000,1996.000000,9.000000,3.000000,320.750000,3.600000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.047500,0.010000,0.040000
50%,1427.500000,25.000000,1999.000000,20.000000,11.000000,1059.000000,11.800000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.080000,0.000000,0.080000,0.060000,0.060000,0.140000,0.060000,0.140000
75%,2143.250000,28.000000,2002.000000,30.000000,23.000000,1999.500000,22.200000,2.000000,2.000000,4.000000,...,0.160000,0.120000,0.290000,0.142500,0.280000,0.180000,0.130000,0.330000,0.170000,0.320000
max,2854.000000,41.000000,2008.000000,38.000000,38.000000,3420.000000,38.000000,31.000000,18.000000,47.000000,...,2.430000,2.650000,2.650000,2.430000,2.650000,5.060000,2.470000,5.060000,5.060000,5.060000


# 3. Exploratory Dataset Analysis

## 📈 Distribusi Usia Pemain
 Bagian ini menganalisis distribusi usia pemain secara keseluruhan dan per liga. Visualisasi histogram akan menunjukkan penyebaran usia, dengan garis vertikal menandakan rata-rata (mean) dan median usia untuk memberikan gambaran pusat data.

In [57]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Menghitung nilai rata-rata (mean) dan median usia pemain.
mean_age = df_cleaned['Age'].mean()
median_age = df_cleaned['Age'].median()

# Membuat histogram untuk distribusi usia pemain di seluruh liga.
fig = px.histogram(
    df_cleaned,
    x="Age",
    nbins=20, # Jumlah 'bin' atau kelompok usia
    marginal="rug",  # Menampilkan garis kecil di bawah untuk setiap poin data
    opacity=0.75,
    color_discrete_sequence=['dodgerblue'], # Warna batang histogram
    title="Distribusi Usia Pemain (Seluruh Liga)",
)

# Menambahkan garis vertikal untuk menunjukkan rata-rata usia.
fig.add_vline(
    x=mean_age,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Rata-rata: {mean_age:.2f}",
    annotation_position="top left"
)

# Menambahkan garis vertikal untuk menunjukkan median usia.
fig.add_vline(
    x=median_age,
    line_dash="dot",
    line_color="red",
    annotation_text=f"Median: {median_age:.2f}",
    annotation_position="top right"
)

# Mengatur posisi judul plot di tengah.
fig.update_layout(title_x=0.5)
fig.show()

Distribusi usia pemain menunjukkan bahwa mayoritas pemain profesional berada di rentang usia produktif (22–28 tahun). Rata-rata dan median yang hampir sama mengindikasikan distribusi seimbang. Di sisi lain, keberadaan pemain muda (<21 tahun) masih minoritas, memperkuat alasan mengapa analisis terhadap kontribusi mereka menjadi penting.

In [58]:
# 1. Mengambil 6 liga dengan jumlah pemain terbanyak.
leagues = df_cleaned['Comp'].value_counts().head(6).index.tolist()

# 2. Membuat subplot dengan tata letak 2 baris dan 3 kolom untuk menampilkan
# distribusi usia per liga.
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=leagues, # Judul untuk setiap subplot adalah nama liga
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

# 3. Melakukan iterasi untuk setiap liga dan menambahkan histogram usia pemain.
for idx, liga in enumerate(leagues):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    data = df_cleaned[df_cleaned['Comp'] == liga]
    mean_age = data['Age'].mean()
    median_age = data['Age'].median()

    # Menambahkan histogram usia untuk liga saat ini.
    fig.add_trace(go.Histogram(
        x=data['Age'],
        nbinsx=20,
        marker_color='dodgerblue',
        opacity=0.75,
        name=liga,
        showlegend=False
    ), row=row, col=col)

    # Menambahkan garis rata-rata usia untuk liga saat ini.
    fig.add_vline(
        x=mean_age, line_dash='dash', line_color='green',
        row=row, col=col,
        annotation_text=f"Rata-rata: {mean_age:.1f}",
        annotation_position="top left",
        annotation_font_size=10
    )

    # Menambahkan garis median usia untuk liga saat ini.
    fig.add_vline(
        x=median_age, line_dash='dot', line_color='red',
        row=row, col=col,
        annotation_text=f"Median: {median_age:.1f}",
        annotation_position="top right",
        annotation_font_size=10
    )

# 4. Mengatur tata letak (layout) plot secara keseluruhan.
fig.update_layout(
    height=650,
    width=1000,
    title="Distribusi Usia Pemain per Liga (dengan Rata-rata & Median)",
    title_x=0.5, # Posisi judul di tengah
    bargap=0.05 # Jarak antar batang histogram
)

# Mengatur label sumbu X dan Y.
fig.update_xaxes(title_text="Usia")
fig.update_yaxes(title_text="Jumlah Pemain")

fig.show()

**📊 Distribusi Usia Pemain per Liga**

Distribusi usia pemain dari lima liga top Eropa menunjukkan pola menarik yang mencerminkan strategi pengelolaan skuad masing-masing liga.

Rata-rata dan median usia pemain di seluruh liga berkisar di angka **24–25 tahun**, menandakan bahwa mayoritas pemain berada di usia puncak performa (prime age).

**🧠 Insight per Liga:**
- 🇫🇷 Ligue 1
Rata-rata usia terendah (~24.1 tahun).
Distribusi condong ke kiri (banyak pemain muda).
Insight: Ligue 1 dikenal sebagai liga pengembang talenta muda. Banyak klub memberi kesempatan besar kepada pemain usia <23 tahun.

- 🇮🇹 Serie A
Rata-rata dan median masih di 25, tetapi distribusi menyebar ke usia lebih tua.
Insight: Serie A memberi tempat untuk pemain senior berpengalaman. Gaya bermain yang lebih taktis memungkinkan pemain veteran tetap bersaing.

- 🇩🇪 Bundesliga dan 🇪🇸 La Liga
Distribusi simetris dan seimbang antara pemain muda dan senior.
Insight: Kedua liga ini menunjukkan manajemen usia yang baik dalam skuad. Memberikan ruang bagi regenerasi sambil tetap mempertahankan pemain berpengalaman.

- 🏴 Premier League
Distribusi padat di usia 25–27.
Insight: EPL cenderung memaksimalkan pemain usia matang. Hal ini sejalan dengan tuntutan fisik dan kompetitif yang sangat tinggi di liga ini.

Distribusi ini bisa ditelusuri lebih dalam dengan membandingkan rata-rata usia antar klub, serta kontribusi gol/assist berdasarkan usia pemain

## 🏃‍♂️ Statistik Pemain Muda
Bagian ini berfokus pada pemain muda (usia <= 21 tahun) untuk menganalisis distribusi mereka di berbagai liga dan klub, serta mengidentifikasi pemain termuda yang aktif.

In [59]:
# Menghitung dan menampilkan jumlah pemain muda (usia <= 21) per liga.
age_threshold = 21 # Batas usia untuk definisi 'pemain muda'
young_players = df_cleaned[df_cleaned['Age'] <= age_threshold]
young_players_per_liga = young_players[['Comp']].value_counts().head(5)

print(young_players_per_liga)

young_players_per_comp = young_players['Comp'].value_counts().head(5).reset_index()
young_players_per_comp.columns = ['Comp', 'Jumlah Pemain Muda']

# Membuat bar chart untuk menunjukkan jumlah pemain muda per liga.
fig = px.bar(
    young_players_per_comp,
    x='Jumlah Pemain Muda',
    y='Comp',
    orientation='h', # Orientasi horizontal
    color='Jumlah Pemain Muda',
    color_continuous_scale='YlGnBu', # Skala warna
    title='Jumlah Pemain Muda per Liga (< 21 Tahun)'
)

fig.update_layout(
    xaxis_title="Jumlah Pemain Muda",
    yaxis_title="Liga",
    yaxis=dict(autorange='reversed'),  # Membalik urutan Y-axis agar bar terbesar di atas
    title_x=0.5
)

fig.show()

Comp              
fr Ligue 1            183
es La Liga            152
it Serie A            132
eng Premier League    127
de Bundesliga          98
Name: count, dtype: int64


In [60]:
# Menghitung dan menampilkan 5 klub dengan jumlah pemain muda terbanyak.
young_players_per_club = young_players[['Squad', 'Comp']].value_counts().head(10)

print(young_players_per_club)

young_players_per_club = young_players['Squad'].value_counts().head(10).reset_index()
young_players_per_club.columns = ['Squad', 'Jumlah Pemain Muda']

# Membuat bar chart untuk menunjukkan 5 klub dengan jumlah pemain muda terbanyak.
fig = px.bar(
    young_players_per_club,
    x='Jumlah Pemain Muda',
    y='Squad',
    orientation='h',
    color='Jumlah Pemain Muda',
    color_continuous_scale='viridis',
    title='Top 5 Klub dengan Jumlah Pemain Muda Terbanyak (< 21 Tahun)'
)

fig.update_layout(
    xaxis_title="Jumlah Pemain Muda",
    yaxis_title="Klub",
    yaxis=dict(autorange='reversed'),
    title_x=0.5
)

fig.show()

Squad           Comp              
Reims           fr Ligue 1            18
Strasbourg      fr Ligue 1            18
Montpellier     fr Ligue 1            14
Valladolid      es La Liga            13
Barcelona       es La Liga            13
Real Madrid     es La Liga            12
Sevilla         es La Liga            11
Tottenham       eng Premier League    11
Lens            fr Ligue 1            11
Eint Frankfurt  de Bundesliga         11
Name: count, dtype: int64


In [61]:
# Menghitung proporsi pemain muda di setiap klub.
young_players = df_cleaned[df_cleaned['Age'] <= age_threshold]

total_per_club = df_cleaned['Squad'].value_counts().reset_index()
total_per_club.columns = ['Squad', 'Total Pemain']

young_per_club = young_players['Squad'].value_counts().reset_index()
young_per_club.columns = ['Squad', 'Pemain Muda']

proporsi_df = pd.merge(total_per_club, young_per_club, on='Squad', how='left')
proporsi_df['Pemain Muda'] = proporsi_df['Pemain Muda'].fillna(0)

proporsi_df['Proporsi Muda (%)'] = 100 * proporsi_df['Pemain Muda'] / proporsi_df['Total Pemain']

top_proporsi = proporsi_df.sort_values('Proporsi Muda (%)', ascending=False).head(10)

# Membuat bar chart untuk menunjukkan 10 klub dengan proporsi pemain muda tertinggi.
fig = px.bar(
    top_proporsi,
    x='Proporsi Muda (%)',
    y='Squad',
    orientation='h',
    color='Proporsi Muda (%)',
    color_continuous_scale='sunsetdark',
    title='Top 10 Klub dengan Proporsi Pemain Muda Tertinggi'
)

fig.update_layout(
    xaxis_title="Proporsi Pemain Muda (%)",
    yaxis_title="Klub",
    yaxis=dict(autorange='reversed'),
    title_x=0.5
)

fig.show()

**Jumlah Pemain Muda di Liga dan Klub**

1. **Dominasi Liga Prancis (Ligue 1) dalam Jumlah Pemain Muda:**

    - Grafik "Jumlah Pemain Muda per Liga (< 21 Tahun)" menunjukkan bahwa Ligue 1 (Liga Prancis) memiliki jumlah pemain muda terbanyak (sekitar 180 pemain) dibandingkan dengan liga-liga besar lainnya seperti La Liga, Serie A, Premier League, dan Bundesliga. Ini mengindikasikan bahwa Liga Prancis adalah liga yang paling banyak mengembangkan atau memberikan kesempatan kepada pemain muda di bawah 21 tahun.

2. **Klub Prancis Memimpin dalam Proporsi dan Jumlah Pemain Muda:**
    - Grafik "Top 5 Klub dengan Jumlah Pemain Muda Terbanyak (< 21 Tahun)" menunjukkan bahwa Strasbourg dan Reims, keduanya klub dari Liga Prancis, memiliki jumlah pemain muda terbanyak (sekitar 17-18 pemain). Ini sejalan dengan temuan di poin 1 dan semakin memperkuat posisi Liga Prancis sebagai liga yang subur untuk pengembangan bakat muda.
    - Grafik "Top 10 Klub dengan Proporsi Pemain Muda Tertinggi" juga menempatkan Strasbourg dan Reims di posisi teratas dalam hal proporsi pemain muda (lebih dari 50%). Ini menunjukkan bahwa di klub-klub ini, pemain muda tidak hanya banyak secara kuantitas, tetapi juga merupakan bagian signifikan dari skuad mereka.

3. Barcelona Menunjukkan Komitmen pada Pemain Muda, Baik dari Jumlah maupun Proporsi:

    - Meskipun tidak berada di posisi teratas dalam jumlah pemain muda, Barcelona (dari La Liga) masuk dalam Top 5 klub dengan jumlah pemain muda terbanyak (sekitar 13-14 pemain) dan juga berada di urutan ketiga dalam proporsi pemain muda tertinggi (sekitar 50%). Hal ini mengindikasikan bahwa Barcelona memiliki komitmen kuat untuk mengintegrasikan dan memberikan kesempatan kepada talenta muda, konsisten dengan filosofi akademi La Masia mereka.

4. Variasi Pendekatan Antar Liga dan Klub:

    - Sementara Liga Prancis secara keseluruhan unggul dalam jumlah pemain muda, liga lain seperti La Liga juga memiliki klub-klub yang sangat berinvestasi pada pemain muda (contohnya Barcelona dan Real Madrid yang muncul di grafik proporsi).

    - Premier League dan Bundesliga, meskipun dianggap sebagai liga top, memiliki jumlah pemain muda yang lebih sedikit secara keseluruhan dibandingkan Ligue 1 dan La Liga, dan tidak ada klub dari Premier League atau Bundesliga yang muncul di Top 5 klub dengan jumlah pemain muda terbanyak. Namun, beberapa klub Bundesliga (RB Leipzig) dan bahkan Eintracht Frankfurt dari Bundesliga muncul di Top 10 proporsi. Ini menunjukkan bahwa tidak semua liga atau klub memiliki pendekatan yang sama dalam pengembangan pemain muda.
  
5. Potensi untuk Rekrutmen Pemain Muda:

    - Bagi klub atau pencari bakat yang ingin merekrut pemain muda dengan potensi tinggi, Liga Prancis, khususnya klub-klub seperti Strasbourg dan Reims, bisa menjadi target utama karena konsentrasi pemain muda yang tinggi.

    - Klub-klub seperti Barcelona dan Real Madrid juga patut diperhatikan karena fokus mereka pada pengembangan pemain muda.

Secara keseluruhan, analisis ini menyoroti bahwa **Liga Prancis adalah lingkungan yang sangat subur untuk pengembangan bakat muda** di antara lima liga top Eropa, baik dari segi jumlah keseluruhan pemain muda maupun proporsi pemain muda di klub-klub tertentu.

Berikut 10 pemain termuda yang debut bersama klub-klub di liga top 5 eropa

In [27]:
# Menampilkan 10 pemain termuda dalam dataset.
youngest_player = df_cleaned.sort_values('Age', ascending=True).reset_index().head(10)
print(youngest_player[['Player', 'Age','Squad','Comp', 'Pos', 'Min']])

              Player   Age          Squad                Comp    Pos   Min
0       Harry Howell  15.0       Brighton  eng Premier League     FW     7
1  Djylian N'Guessan  15.0  Saint-Étienne          fr Ligue 1     FW   181
2        Mikey Moore  16.0      Tottenham  eng Premier League     FW   366
3     Ayyoub Bouaddi  16.0          Lille          fr Ligue 1     MF  1151
4      Mohamed Meité  16.0         Rennes          fr Ligue 1     FW   544
5         Faik Sakar  16.0     RB Leipzig       de Bundesliga     MF     1
6    Kyllian Antonio  16.0           Lens          fr Ligue 1  DF,MF   139
7       David Otorbi  16.0       Valencia          es La Liga     MF    14
8        Viggo Gebel  16.0     RB Leipzig       de Bundesliga     MF    14
9      Ibrahim Mbaye  16.0      Paris S-G          fr Ligue 1  FW,MF   340


In [62]:
# Menampilkan 3 pemain termuda dari setiap liga.
youngest_players_per_liga = (
    df_cleaned.sort_values('Age')
    .groupby('Comp')
    .head(3)
    .reset_index(drop=True)
)

print(youngest_players_per_liga[['Player', 'Age', 'Squad', 'Comp', 'Pos', 'Min']].sort_values('Comp').reset_index(drop=True))


                Player   Age          Squad                Comp    Pos   Min
0           Faik Sakar  16.0     RB Leipzig       de Bundesliga     MF     1
1          Viggo Gebel  16.0     RB Leipzig       de Bundesliga     MF    14
2    Kacper Koscierski  17.0         Bochum       de Bundesliga     DF     7
3         Harry Howell  15.0       Brighton  eng Premier League     FW     7
4          Mikey Moore  16.0      Tottenham  eng Premier League     FW   366
5      Shumaira Mheuka  16.0        Chelsea  eng Premier League     MF     1
6         David Otorbi  16.0       Valencia          es La Liga     MF    14
7          Héctor Fort  17.0      Barcelona          es La Liga     DF   585
8     Arturo Rodríguez  17.0     Las Palmas          es La Liga     FW    21
9    Djylian N'Guessan  15.0  Saint-Étienne          fr Ligue 1     FW   181
10      Ayyoub Bouaddi  16.0          Lille          fr Ligue 1     MF  1151
11       Mohamed Meité  16.0         Rennes          fr Ligue 1     FW   544

## ⏱️ Analisis Menit Bermain dan Tingkat Starter
Bagian ini menganalisis seberapa banyak menit bermain yang didapatkan pemain muda
dan seberapa sering mereka menjadi starter. Ini menunjukkan tingkat kepercayaan pelatih terhadap pemain muda dan potensi mereka sebagai pemain inti di masa depan.


In [63]:
# Menghitung rata-rata menit bermain untuk pemain muda.
avg_minutes_young = young_players['Min'].mean()
print(f"Rata-rata menit bermain pemain muda (≤ {age_threshold:.1f} tahun): {avg_minutes_young:.2f} menit")

# Menghitung rata-rata menit bermain pemain muda per liga.
avg_min_per_league = young_players.groupby('Comp')['Min'].mean().round(2).sort_values(ascending=False)
print(avg_min_per_league)

# Membuat bar chart untuk rata-rata menit bermain pemain muda per liga.
fig = px.bar(
    avg_min_per_league.reset_index(),
    x='Min',
    y='Comp',
    orientation='h',
    text='Min',
    color = 'Min',
    color_continuous_scale = 'Emrld',
    title='Rata-rata Menit Bermain Pemain Muda per Liga'
)

fig.update_layout(
    yaxis=dict(title='Liga'),
    xaxis=dict(title='Rata-rata Menit'),
    yaxis_autorange='reversed',
    title_x=0.5
)

fig.show()

Rata-rata menit bermain pemain muda (≤ 21.0 tahun): 688.67 menit
Comp
eng Premier League    746.83
de Bundesliga         702.61
es La Liga            692.19
fr Ligue 1            684.65
it Serie A            623.90
Name: Min, dtype: float64


In [64]:
# Menghitung rata-rata menit bermain pemain muda per klub.
avg_min_per_club = young_players.groupby('Squad')['Min'].mean().round(2).sort_values(ascending=False)

# Membuat bar chart untuk 10 klub dengan rata-rata menit bermain pemain muda tertinggi.
fig = px.bar(
    avg_min_per_club.head(10).reset_index(),
    x='Min',
    y='Squad',
    orientation='h',
    color = 'Min',
    color_continuous_scale='inferno_r',
    text='Min',
    title='Top 10 Klub dengan Rata-rata Menit Bermain Pemain Muda Tertinggi'
)

fig.update_layout(
    yaxis=dict(title='Klub'),
    xaxis=dict(title='Rata-rata Menit'),
    yaxis_autorange='reversed',
    title_x=0.5
)

fig.show()

1. **Menit Bermain Tidak Selalu Sejalan dengan Jumlah/Proporsi Pemain Muda**:

    - Meskipun Liga Prancis sebelumnya menonjol dalam jumlah dan proporsi pemain muda, grafik menit bermain menunjukkan gambaran yang sedikit berbeda. Rata-rata menit bermain pemain muda di Liga Prancis (684.65 menit) berada di bawah rata-rata keseluruhan (688.67 menit) dan juga di bawah Premier League, Bundesliga, dan La Liga. Ini menyiratkan bahwa meskipun banyak pemain muda di Ligue 1, mereka mungkin tidak selalu mendapatkan waktu bermain yang signifikan secara rata-rata di seluruh liga.
    - Namun, Strasbourg dan Paris Saint-Germain (PSG) dari Ligue 1 masih muncul di Top 10 klub dengan rata-rata menit bermain tertinggi. Ini menunjukkan bahwa di beberapa klub Prancis, pemain muda memang mendapatkan banyak kesempatan, tetapi mungkin ada banyak klub lain di liga yang kurang memberikan menit bermain.

2. **Premier League Memimpin dalam Rata-rata Menit Bermain Pemain Muda di Tingkat Liga, Namun Klub-klub Tertentu yang Menonjol**:

    - Liga Premier memiliki rata-rata menit bermain pemain muda tertinggi (746.83 menit) di antara lima liga top. Ini menunjukkan bahwa secara keseluruhan, klub-klub di Premier League cenderung lebih banyak memberikan kesempatan bermain kepada pemain muda dibandingkan liga lainnya.
    - Menariknya, Ipswich Town (mungkin dari liga Championship Inggris atau tim promosi baru ke Premier League), Nott'ham Forest, Newcastle Utd, dan Brighton dari Premier League semuanya masuk dalam Top 10 klub dengan rata-rata menit bermain pemain muda tertinggi. Hal ini menegaskan komitmen beberapa klub Premier League dalam memberikan menit bermain signifikan kepada pemain muda.

3. **La Liga dan Bundesliga Memberikan Kesempatan Menit Bermain yang Konsisten**:

    - La Liga (692.19 menit) dan Bundesliga (702.61 menit) memiliki rata-rata menit bermain pemain muda yang lebih tinggi dari rata-rata keseluruhan. Ini menunjukkan bahwa kedua liga ini juga memberikan kesempatan yang cukup baik bagi pemain muda untuk mendapatkan pengalaman bermain.
    - Celta Vigo, Atlético Madrid, dan Barcelona dari La Liga muncul di daftar Top 10 klub dengan menit bermain tertinggi. Ini mengindikasikan bahwa tidak hanya Barcelona yang memberikan menit bermain, tetapi juga klub-klub lain di La Liga.

4. **Serie A Memberikan Menit Bermain Paling Sedikit Secara Rata-rata**:

    - Serie A memiliki rata-rata menit bermain pemain muda terendah (623.90 menit). Ini menunjukkan bahwa secara keseluruhan, pemain muda di Liga Italia mungkin menghadapi tantangan lebih besar dalam mendapatkan waktu bermain yang konsisten dibandingkan liga-liga top lainnya. Meskipun demikian, Como yang baru promosi dari Serie B, masuk dalam top 10 menit bermain tertinggi, ini mungkin disebabkan skuad mereka banyak berisi pemain muda dan minim pengalaman.

5. **Klub-Klub Unggulan Memberikan Waktu Bermain yang Signifikan**:

    - Kehadiran klub-klub besar seperti Barcelona, Atlético Madrid, dan Paris Saint-Germain di daftar Top 10 klub dengan rata-rata menit bermain pemain muda tertinggi sangat signifikan. Ini menunjukkan bahwa bahkan di klub-klub dengan ambisi tinggi dan persaingan ketat, ada komitmen untuk mengintegrasikan dan memberikan pengalaman bermain yang berharga bagi talenta muda.
  
Meskipun Liga Prancis unggul dalam jumlah dan proporsi pemain muda, Premier League secara keseluruhan memberikan rata-rata menit bermain terbanyak bagi pemain muda. Namun, ada klub-klub spesifik di setiap liga (seperti Ipswich Town, Celta Vigo, Strasbourg, Atlético Madrid, Nott'ham Forest, Newcastle Utd, Brighton, Barcelona, Como, dan PSG) yang benar-benar berkomitmen untuk memberikan pengalaman bermain yang signifikan kepada talenta muda mereka. Ini adalah indikator penting untuk mengidentifikasi lingkungan terbaik bagi perkembangan pemain muda di sepak bola profesional.

In [65]:
# Mengambil 10 pemain muda dengan menit bermain terbanyak.
top_10_min_play_young_player = (
    young_players[['Player', 'Min', 'Age', 'Squad', 'Comp']]
    .sort_values('Min', ascending=False)
    .head(10)
    .reset_index(drop=True)
)

print(top_10_min_play_young_player)

# Membuat bar chart untuk 10 pemain muda dengan menit bermain terbanyak.
fig = px.bar(
    top_10_min_play_young_player,
    x='Min',
    y='Player',
    orientation='h',
    color='Min',
    color_continuous_scale = 'bupu',
    text='Min',
    title='Top 10 Pemain Muda dengan Menit Bermain Terbanyak'
)
fig.update_layout(yaxis=dict(autorange='reversed'), title_x=0.5)
fig.show()

               Player   Min   Age          Squad                Comp
0        Milos Kerkez  3336  20.0    Bournemouth  eng Premier League
1  Cristhian Mosquera  3319  20.0       Valencia          es La Liga
2         Zion Suzuki  3314  21.0          Parma          it Serie A
3     Bart Verbruggen  3240  21.0       Brighton  eng Premier League
4      Omar El Hilali  3153  20.0       Espanyol          es La Liga
5        Levi Colwill  3149  21.0        Chelsea  eng Premier League
6      Illia Zabarnyi  3109  21.0    Bournemouth  eng Premier League
7       Diego Coppola  2927  20.0  Hellas Verona          it Serie A
8    Mateus Fernandes  2909  20.0    Southampton  eng Premier League
9               Pedri  2879  21.0      Barcelona          es La Liga


In [66]:
# Membandingkan total menit bermain dengan total '90s' (jumlah pertandingan penuh 90 menit).
compare_minutes = (
    young_players[['Player', 'Min', '90s', 'Squad', 'Comp']]
    .sort_values('Min', ascending=False)
    .head(10)
    .reset_index(drop=True)
)

fig = go.Figure()

# Bar chart untuk total menit bermain.
fig.add_trace(go.Bar(
    x=compare_minutes['Player'],
    y=compare_minutes['Min'],
    name='Total Menit',
    marker_color='#97B067'  # Hijau tua
))

# Bar chart untuk total menit berdasarkan '90s' (jumlah pertandingan 90 menit).
fig.add_trace(go.Bar(
    x=compare_minutes['Player'],
    y=compare_minutes['90s'] * 90,
    name='(90s × 90 menit)',
    marker_color='#E3DE61'  # Hijau muda
))

fig.update_layout(
    barmode='group', # Batang dikelompokkan
    title='Perbandingan Total Menit vs 90s (Pemain Muda)',
    xaxis_title='Pemain',
    yaxis_title='Menit Bermain',
    title_x=0.5,
    legend=dict(x=0.5, xanchor='center', y=1.1, orientation='h')
)

fig.show()

In [67]:
young_players_played = young_players.sort_values('MP', ascending=False)
young_players_start = young_players.sort_values('Starts', ascending=False)

top_young = young_players.sort_values('MP', ascending=False).head(10)

# Membuat bar chart untuk membandingkan jumlah pertandingan dimainkan (MP) dan jumlah starter (Starts).
fig = go.Figure()
fig.add_trace(go.Bar(x=top_young['Player'], y=top_young['MP'], name='Total Main', marker_color='#00879E'))
fig.add_trace(go.Bar(x=top_young['Player'], y=top_young['Starts'], name='Starter', marker_color='#FFAB5B'))

fig.update_layout(
    barmode='group',
    title='Pemain Muda dengan Jumlah Pertandingan Terbanyak (MP vs Starts)',
    xaxis_title='Pemain',
    yaxis_title='Jumlah Pertandingan',
    title_x=0.5
)
fig.show()

**1. Dominasi Milos Kerkez dalam Menit Bermain dan Jumlah Pertandingan:**
* Grafik "Top 10 Pemain Muda dengan Menit Bermain Terbanyak" dan "Perbandingan Total Menit vs 90s (Pemain Muda)" dengan jelas menunjukkan bahwa Milos Kerkez adalah pemain muda dengan menit bermain terbanyak (3336 menit). Ini adalah jumlah yang sangat signifikan untuk pemain muda.
* Grafik "Pemain Muda dengan Jumlah Pertandingan Terbanyak (MP vs Starts)" (newplot (9).png) juga menempatkan Milos Kerkez di posisi teratas dalam total pertandingan yang dimainkan (sekitar 38-39 pertandingan) dan juga sebagai *starter* (sekitar 33-34 pertandingan). Hal ini menunjukkan konsistensi dan kepercayaan yang tinggi dari timnya kepadanya.

**2. Tingginya Kepercayaan Pelatih pada Pemain Muda Tertentu:**
* Beberapa pemain muda secara konsisten mendapatkan menit bermain yang sangat tinggi, mendekati atau bahkan melebihi 3000 menit. Selain Milos Kerkez, ada Cristhian Mosquera, Zion Suzuki, Bart Verbruggen, Omar El Hilali, dan Levi Colwill yang semuanya bermain lebih dari 3000 menit.
* Grafik perbandingan total menit vs 90s menunjukkan bahwa sebagian besar pemain di daftar ini memiliki total menit bermain yang sangat dekat dengan jumlah menit jika mereka bermain penuh 90 menit di setiap pertandingan (90s x 90 menit). Ini menandakan bahwa mereka bukan hanya sering bermain, tetapi juga sering bermain penuh atau hampir penuh dalam pertandingan yang mereka ikuti, menunjukkan peran kunci dalam tim.

**3. Pentingnya Konsistensi dan Peran Starter:**

* Pemain seperti Milos Kerkez, Cristhian Mosquera, dan Zion Suzuki tidak hanya sering bermain, tetapi juga sering menjadi *starter*. Milos Kerkez, misalnya, memiliki rasio *starter* yang sangat tinggi dibandingkan total pertandingannya. Ini menunjukkan bahwa mereka bukan hanya mengisi slot di skuad, tetapi benar-benar menjadi pilihan utama di posisi mereka.
* Pemain lain seperti Pedri, meskipun total menit bermainnya sedikit lebih rendah dibandingkan yang teratas, masih menunjukkan rasio *starter* yang kuat, menegaskan perannya sebagai pemain kunci.

**4. Keberagaman Posisi dan Liga:**

* Meskipun grafik tidak secara spesifik menyebutkan posisi atau liga asal masing-masing pemain, fakta bahwa ada pemain muda yang mengumpulkan menit bermain sebanyak ini menunjukkan bahwa ada talenta muda yang sudah dipercaya untuk bermain di level profesional tertinggi.
* Kehadiran pemain seperti Pedri (Barcelona) menegaskan kembali bahwa klub-klub besar pun memberikan kesempatan besar kepada talenta muda yang luar biasa.

**5. Indikator Potensi dan Perkembangan:**

* Pemain-pemain dalam daftar ini, terutama yang mengumpulkan menit bermain di atas 3000, adalah indikator kuat dari potensi besar mereka. Jumlah menit bermain yang tinggi pada usia muda menunjukkan bahwa mereka mampu bersaing di level senior, memiliki daya tahan fisik yang baik, dan konsisten dalam performa.
* Bagi klub, data ini bisa menjadi dasar untuk mengidentifikasi talenta yang paling siap dan paling banyak berkembang melalui pengalaman bermain langsung.

Secara keseluruhan, grafik-grafik ini memberikan gambaran yang sangat baik tentang pemain muda mana yang paling banyak mendapatkan pengalaman bermain di lapangan, sebuah faktor kunci dalam perkembangan mereka menuju potensi penuh. Milos Kerkez secara khusus menonjol sebagai contoh utama dari pemain muda yang telah dipercaya sepenuhnya oleh timnya.

In [71]:
# Menghitung rasio starter (Starts / MP) untuk pemain muda yang bermain minimal 20 pertandingan.
young_players['starter_ratio'] = young_players['Starts'] / young_players['MP']
young_players_filtered = young_players[young_players['MP'] >= 20]  # Filter pemain yang cukup sering main
top_starter = young_players_filtered.sort_values(['starter_ratio', 'MP'], ascending=False).head(10)

# Membuat bar chart untuk rasio starter pemain muda.
fig = px.bar(
    top_starter,
    x='Player',
    y='starter_ratio',
    color='starter_ratio',
    color_continuous_scale='viridis',
    title='Rasio Starter Pemain Muda (Minimal 20 Pertandingan)',
    labels={'starter_ratio': 'Rasio Starter'},
    hover_data=['Squad', 'MP', 'Starts', '90s']
)
fig.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig.update_layout(title_x=0.5)
fig.show()

In [69]:
# Menghitung rata-rata rasio starter pemain muda per liga.
starter_by_league = young_players.groupby('Comp')[['MP', 'Starts']].sum()
starter_by_league['Starter Ratio'] = starter_by_league['Starts'] / starter_by_league['MP']
starter_by_league = starter_by_league.sort_values('Starter Ratio', ascending=False).reset_index()

# Membuat bar chart untuk rasio starter pemain muda per liga.
px.bar(
    starter_by_league,
    x='Comp',
    y='Starter Ratio',
    color='Starter Ratio',
    color_continuous_scale='plasma',
    title='Tingkat Starter Pemain Muda per Liga',
    labels={'Starter Ratio': 'Rasio Starter'}
).show()

1. Keberadaan Pemain Muda "Anti-Rotasi" dengan Peran Kunci Absolut:
    * Grafik "Rasio Starter Pemain Muda (Minimal 20 Pertandingan)" adalah yang paling mencolok. Semua pemain yang terdaftar di sini (Milos Kerkez, Cristhian Mosquera, Zion Suzuki, Omar El Hilali, Bart Verbruggen, Levi Colwill, Andrey Santos, Ismaël Doukouré, Guillaume Restes, dan Emanuel Emegha) memiliki rasio starter 1.00.

    * Ini berarti kesepuluh pemain muda ini, dalam minimal 20 pertandingan yang mereka ikuti, selalu menjadi starter. Ini adalah indikator yang sangat kuat bahwa mereka bukan hanya talenta menjanjikan, tetapi juga pilar utama dalam tim mereka, bahkan pada usia muda. Mereka hampir tidak pernah dicadangkan, menunjukkan tingkat kepercayaan dan konsistensi performa yang luar biasa.

2. Konfirmasi Dominasi Milos Kerkez dan Zion Suzuki sebagai Pemain Inti:
    * Kehadiran Milos Kerkez dan Zion Suzuki di daftar ini (dengan rasio starter 1.00 pada minimal 20 pertandingan) memperkuat status mereka sebagai salah satu pemain muda paling penting dan diandalkan di sepak bola profesional saat ini. Mereka adalah contoh sempurna dari pemain muda yang sudah menjadi pilihan utama yang tak tergantikan.

3. La Liga Tetap Memimpin dalam Memberikan Peran Starter kepada Pemain Muda:
    * Grafik "Tingkat Starter Pemain Muda per Liga" konsisten menunjukkan bahwa La Liga memiliki rasio starter pemain muda tertinggi (sekitar 0.60 atau 60%). Ini memperkuat poin bahwa klub-klub di Spanyol secara umum lebih cenderung untuk memberikan peran starter kepada pemain muda mereka dibandingkan dengan liga-liga top lainnya.

    * Ini menunjukkan lingkungan yang sangat mendukung bagi pemain muda untuk mendapatkan pengalaman bermain di level tertinggi sebagai pemain inti.

4. Konsistensi dalam Kesempatan Starter di Premier League dan Ligue 1:
    * Premier League dan Ligue 1 tetap berada di posisi kedua dan ketiga dengan rasio starter yang sangat dekat (sekitar 0.59). Ini menunjukkan bahwa meskipun La Liga sedikit lebih unggul, kedua liga ini juga sangat proaktif dalam memberikan kesempatan starter kepada pemain muda mereka.

5. Bundesliga dan Serie A Masih di Bawah Rata-rata dalam Rasio Starter:
    * Bundesliga dan Serie A memiliki rasio starter yang lebih rendah (sekitar 0.56-0.57). Meskipun mereka tetap memberikan kesempatan, pemain muda di liga-liga ini mungkin lebih sering memulai dari bangku cadangan atau digunakan sebagai bagian dari rotasi dibandingkan menjadi starter reguler di banyak pertandingan.

**Kesimpulan Sementara**
- La Liga adalah liga yang paling konsisten dalam memberikan peran starter kepada pemain muda.

- Ligue 1 menonjol dalam hal jumlah dan proporsi pemain muda, menunjukkan keberanian dalam memperkenalkan talenta baru.

- Premier League memberikan menit bermain terbanyak secara rata-rata kepada pemain muda, menunjukkan lingkungan yang kompetitif namun juga memberikan kepercayaan.

- Ada sebuah "elite" pemain muda (seperti Milos Kerkez, Zion Suzuki, Cristhian Mosquera, Omar El Hilali, Bart Verbruggen, Levi Colwill, dan lainnya yang muncul di grafik 1.00 rasio starter untuk 20+ pertandingan) yang telah berhasil menjadi pemain inti yang tak tergantikan di tim mereka pada usia yang sangat muda. Mereka adalah indikator utama dari talenta luar biasa yang sudah mencapai kematangan profesional.

- Analisis ini memberikan gambaran yang sangat mendalam tentang bagaimana pemain muda digunakan di liga-liga top Eropa, mulai dari sekadar keberadaan, menit bermain, hingga peran penting sebagai starter yang tak tergantikan.

## ⚽ Performa Pemain Muda: Gol dan Assist
Bagian ini menganalisis kontribusi gol dan assist pemain muda secara keseluruhan, serta membandingkan gol aktual dengan Expected Goals (xG) dan assist aktual dengan Expected Assisted Goals (xAG) untuk mengukur efisiensi mereka.

In [72]:
# Menghitung total gol, assist, dan G+A (Gol + Assist) untuk semua pemain.
total_goals = df_cleaned['Gls'].sum()
total_assists = df_cleaned['Ast'].sum()
total_ga = df_cleaned['Gls'].add(df_cleaned['Ast']).sum()

# Menghitung total gol, assist, dan G+A untuk pemain muda.
young_goals = young_players['Gls'].sum()
young_assists = young_players['Ast'].sum()
young_ga = young_players['Gls'].add(young_players['Ast']).sum()

# Menghitung persentase kontribusi pemain muda terhadap total kontribusi.
percent_goals = 100 * young_goals / total_goals
percent_assists = 100 * young_assists / total_assists
percent_ga = 100 * young_ga / total_ga

# Mencetak hasil kontribusi pemain muda.
print(f"Batas usia pemain muda: ≤ {age_threshold:.2f} tahun")
print(f"Kontribusi Gol oleh pemain muda: {young_goals} dari {total_goals} ({percent_goals:.2f}%)")
print(f"Kontribusi Assist oleh pemain muda: {young_assists} dari {total_assists} ({percent_assists:.2f}%)")
print(f"Kontribusi Total G+A oleh pemain muda: {young_ga} dari {total_ga} ({percent_ga:.2f}%)")

# Data untuk pie chart.
goals_data = [young_goals, total_goals - young_goals]
assists_data = [young_assists, total_assists - young_assists]
ga_data = [young_ga, total_ga - young_ga]

labels = ['Pemain Muda', 'Pemain Lain']
colors = ['#F05A7E', '#125B9A']

# Membuat subplot dengan 3 pie chart untuk visualisasi kontribusi.
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
    subplot_titles=["Kontribusi Gol", "Kontribusi Assist", "Kontribusi G+A"]
)

# Menambahkan pie chart untuk kontribusi gol.
fig.add_trace(go.Pie(
    labels=labels,
    values=goals_data,
    name="Gol",
    marker=dict(colors=colors),
    hole=0.5, # Membuat donat chart
    pull=[0.1, 0]  # Sedikit memisahkan bagian 'Pemain Muda'
), 1, 1)

# Menambahkan pie chart untuk kontribusi assist.
fig.add_trace(go.Pie(
    labels=labels,
    values=assists_data,
    name="Assist",
    marker=dict(colors=colors),
    hole=0.5,
    pull=[0.1, 0]
), 1, 2)

# Menambahkan pie chart untuk kontribusi G+A.
fig.add_trace(go.Pie(
    labels=labels,
    values=ga_data,
    name="G+A",
    marker=dict(colors=colors),
    hole=0.5,
    pull=[0.1, 0]
), 1, 3)

# Mengatur tata letak plot.
fig.update_layout(
    title_text="Kontribusi Gol & Assist oleh Pemain Muda (Usia ≤ 21 Tahun)",
    annotations=[dict(text='Gol', x=0.15, y=0.5, font_size=14, showarrow=False),
                 dict(text='Assist', x=0.5, y=0.5, font_size=14, showarrow=False),
                 dict(text='G+A', x=0.85, y=0.5, font_size=14, showarrow=False)]
)

fig.show()

Batas usia pemain muda: ≤ 21.00 tahun
Kontribusi Gol oleh pemain muda: 672 dari 4801 (14.00%)
Kontribusi Assist oleh pemain muda: 534 dari 3425 (15.59%)
Kontribusi Total G+A oleh pemain muda: 1206 dari 8226 (14.66%)


1. **Kontribusi Ofensif yang Signifikan dari Pemain Muda:**
    - Meskipun pemain muda (usia ≤ 21 tahun) hanya menyumbang sebagian kecil dari total gol, assist, dan gabungan gol+assist, angka ini tetaplah signifikan mengingat usia mereka.
    - Pemain muda berkontribusi 14% dari total gol, 15.6% dari total assist, dan 14.7% dari total Gol & Assist (G+A).
2. **Pemain Muda Lebih Menonjol dalam Memberikan Assist:**
    - Menariknya, kontribusi pemain muda dalam assist (15.6%) sedikit lebih tinggi daripada kontribusi mereka dalam gol (14%). Ini bisa mengindikasikan bahwa banyak pemain muda yang berposisi sebagai playmaker, winger yang mengirimkan umpan silang, atau gelandang kreatif yang fokus pada penciptaan peluang, dibandingkan sebagai pencetak gol utama.
    - Hal ini juga bisa berarti bahwa mereka mungkin lebih sering terlibat dalam fase pembangunan serangan dan distribusi bola, menunjukkan visi dan kemampuan umpan yang baik.
3. **Potensi Besar untuk Pertumbuhan di Masa Depan:**
    - Angka kontribusi ini, meskipun bukan mayoritas, sangat menjanjikan. Mengingat bahwa mereka masih dalam tahap pengembangan, kemampuan mereka untuk secara kolektif menyumbangkan hampir 15% dari total ofensif adalah bukti potensi besar mereka.
    - Seiring bertambahnya pengalaman, kematangan fisik, dan pemahaman taktis, diharapkan kontribusi mereka akan terus meningkat di tahun-tahun mendatang.
4. **Ketergantungan Tim pada Pemain Senior/Mapan:**
    - Mayoritas kontribusi ofensif (sekitar 85%) masih berasal dari "Pemain Lain" (pemain di atas 21 tahun). Ini wajar, mengingat pengalaman, kematangan fisik, dan posisi kunci yang sering diisi oleh pemain senior.
    - Namun, angka ini juga menyoroti bahwa meskipun ada talenta muda yang menjanjikan, tim-tim top masih sangat mengandalkan output ofensif dari pemain yang lebih berpengalaman.
5. **Pentingnya Investasi pada Pengembangan Pemain Muda:**
    - Kontribusi ini, meskipun minoritas, menunjukkan bahwa investasi pada pemain muda sangat berharga. Pemain-pemain ini memberikan nilai tambah yang nyata pada statistik ofensif tim mereka, dan mereka adalah aset masa depan yang akan menjadi tulang punggung tim di kemudian hari.

Secara keseluruhan, grafik ini memberikan gambaran jelas bahwa pemain muda telah mulai memberikan dampak nyata dalam aspek ofensif sepak bola, terutama dalam menciptakan peluang (assist). Ini menggarisbawahi pentingnya terus mendukung dan mengembangkan talenta-talenta ini untuk masa depan olahraga.

In [73]:
# Menampilkan 10 pemain muda dengan gol terbanyak, memisahkan gol dari penalti (PK) dan non-penalti (G-PK).
young_players = df_cleaned[df_cleaned['Age'] <= age_threshold].sort_values('Gls', ascending=False)
data = young_players[['Player', 'Gls', 'G-PK', 'PK', 'Age', 'Squad', 'Min']].head(10).reset_index(drop=True)

fig = go.Figure()

# Bar chart untuk total gol.
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['Gls'],
    name='Total Gol',
    marker_color='#2B3467',
    offsetgroup=0,
    hovertext=data.apply(lambda row: f"Usia: {row['Age']}<br>Klub: {row['Squad']}<br>Menit: {row['Min']}\n", axis=1)
))

# Bar chart untuk gol non-penalti (bagian bawah stack).
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['G-PK'],
    name='Gol Non-Penalti',
    marker_color='#BAD7E9',
    offsetgroup=1,
    base=0
))

# Bar chart untuk gol penalti (ditumpuk di atas non-penalti).
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['PK'],
    name='Gol with Penalti',
    marker_color='#EB455F',
    offsetgroup=1,
    base=data['G-PK']
))

fig.update_layout(
    barmode='group', # Batang dikelompokkan
    title='Top 10 Pemain Muda: Perbandingan Total Gol vs (Gol Non-Penalti + Penalti)',
    xaxis_title='Pemain',
    yaxis_title='Jumlah Gol',
    title_x=0.5,
    legend=dict(x=0.5, xanchor='center', y=1.1, orientation='h')
)

fig.show()

In [74]:
# Menampilkan 5 pencetak gol terbanyak dari pemain muda di setiap liga.
leagues = young_players['Comp'].value_counts().head(6).index.tolist()

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f"{liga}" for liga in leagues],
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

for idx, liga in enumerate(leagues):
    row = idx // 3 + 1
    col = idx % 3 + 1

    data = young_players[young_players['Comp'] == liga].sort_values('Gls', ascending=False).head(5)

    fig.add_trace(go.Bar(
        x=data['Gls'],
        y=data['Player'],
        orientation='h',
        name=liga,
        marker_color='#387ADF',
        text=data['Gls'],
        textposition='auto'
    ), row=row, col=col)

    fig.update_yaxes(autorange='reversed', row=row, col=col)

fig.update_layout(
    height=600,
    width=1000,
    title_text=f"Top 5 Pencetak Gol Pemain Muda (≤ {age_threshold:.1f} Tahun) di Tiap Liga",
    showlegend=False,
    title_x=0.5
)

fig.show()

1. Keberagaman Distribusi Top Skorer Muda di Tiap Liga:
    - Ligue 1 menonjol dengan dua pemain teratas (Emanuel Emegha dan Bradley Barcola) yang sama-sama mencetak 14 gol. Hal ini menunjukkan bahwa Ligue 1 memiliki pemain muda yang sangat produktif dalam mencetak gol.
    - Bundesliga juga memiliki pencetak gol muda yang sangat produktif, terutama Benjamin Šeško dengan 13 gol dan Jamal Musiala dengan 12 gol.
    - Premier League dan La Liga memiliki pencetak gol muda terbanyak di angka 12 (Liam Delap) dan 11 (Thierno Barry) secara berturut-turut.
    - Serie A memiliki angka pencetak gol muda terendah di antara lima liga, dengan Assane Diao dan Santiago Castro masing-masing 8 gol. Ini konsisten dengan temuan sebelumnya mengenai menit bermain yang lebih rendah di Serie A, yang mungkin memengaruhi kontribusi gol.

2. Efektivitas Gol Non-Penalti yang Tinggi dari Pencetak Gol Muda:
    - Grafik "Top 10 Pemain Muda: Perbandingan Total Gol vs (Gol Non-Penalti + Penalti)" menunjukkan bahwa sebagian besar gol yang dicetak oleh pemain muda di daftar ini adalah gol non-penalti.
    - Emanuel Emegha dan Bradley Barcola bahkan mencetak 14 gol tanpa penalti. Ini sangat impresif karena menunjukkan kemampuan open play mereka dalam mencetak gol.
    - Benjamin Šeško, Mika Biereth, dan Liam Delap adalah beberapa pemain yang memiliki kontribusi penalti yang signifikan, namun gol non-penalti mereka tetap tinggi, menunjukkan kemampuan mencetak gol dari berbagai situasi.
    - Pemain seperti Lucas Stassin, Jamal Musiala, Thierno Barry, Paul Nebel, dan Andrey Santos memiliki semua gol mereka berasal dari open play (gol non-penalti) di daftar Top 10 ini, yang menunjukkan kualitas mereka sebagai pencetak gol dari permainan terbuka.

3. Pemain Muda Sudah Menjadi Ancaman Gol Utama:
    - Fakta bahwa beberapa pemain muda mampu mencetak 12-14 gol dalam satu musim menunjukkan bahwa mereka sudah menjadi ancaman gol yang signifikan bagi tim mereka. Angka ini merupakan kontribusi yang sangat berharga untuk klub.
    - Ini juga menggarisbawahi potensi mereka untuk menjadi striker atau playmaker kelas dunia di masa depan.

4. Kualitas Serangan Muda di Ligue 1 dan Bundesliga:
    - Kedua liga ini, terutama Ligue 1, menampilkan jumlah pencetak gol muda terbanyak yang mencapai angka belasan. Hal ini menunjukkan kekuatan ofensif yang signifikan dari talenta muda di liga tersebut.

5. Pentingnya Kontribusi Gol dari Pemain Muda:
    - Mengingat bahwa grafik sebelumnya menunjukkan pemain muda berkontribusi sekitar 14% dari total gol, detail top skorer ini memperlihatkan siapa saja individu yang mendorong kontribusi tersebut. Ini adalah bukti nyata bahwa pemain muda tidak hanya bermain, tetapi juga memiliki dampak langsung pada hasil pertandingan melalui gol-gol mereka.

Secara keseluruhan, analisis ini menyoroti bahwa ada beberapa pemain muda yang sangat produktif dalam mencetak gol di berbagai liga top Eropa, dengan sebagian besar gol mereka berasal dari permainan terbuka. Ligue 1 dan Bundesliga secara khusus menonjol dalam memiliki pencetak gol muda terbanyak di angka belasan. Ini adalah indikator kuat dari bakat ofensif yang sedang berkembang di sepak bola global.

In [75]:
# Menampilkan 5 pencetak assist terbanyak dari pemain muda di setiap liga.
leagues = young_players['Comp'].value_counts().head(6).index.tolist()

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f"{liga}" for liga in leagues],
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

for idx, liga in enumerate(leagues):
    row = idx // 3 + 1
    col = idx % 3 + 1

    data = young_players[young_players['Comp'] == liga].sort_values('Ast', ascending=False).head(5)

    fig.add_trace(go.Bar(
        x=data['Ast'],
        y=data['Player'],
        orientation='h',
        name=liga,
        marker_color='#FBA834',
        text=data['Ast'],
        textposition='auto'
    ), row=row, col=col)

    fig.update_yaxes(autorange='reversed', row=row, col=col)

fig.update_layout(
    height=600,
    width=1000,
    title_text=f"Top 5 Pencetak Assist Pemain Muda (≤ {age_threshold:.1f} Tahun) per Liga",
    showlegend=False,
    title_x=0.5
)

fig.show()

In [76]:
# Menampilkan 5 pemain muda dengan kombinasi Gol + Assist (G+A) terbanyak di setiap liga.
young_players = df_cleaned[df_cleaned['Age'] <= age_threshold].copy()
young_players['G+A'] = young_players['Gls'] + young_players['Ast']

leagues = young_players['Comp'].value_counts().head(6).index.tolist()

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f"{liga}" for liga in leagues],
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

for idx, liga in enumerate(leagues):
    row = idx // 3 + 1
    col = idx % 3 + 1

    data = (
        young_players[young_players['Comp'] == liga]
        .sort_values('G+A', ascending=False)
        .head(5)
    )

    fig.add_trace(go.Bar(
        x=data['G+A'],
        y=data['Player'],
        orientation='h',
        name=liga,
        marker_color='#512B81',
        text=data['G+A'],
        textposition='auto',
        hovertext=[
            f"Gol: {g} | Assist: {a} | Klub: {s}"
            for g, a, s in zip(data['Gls'], data['Ast'], data['Squad'])
        ],
        hoverinfo='text'
    ), row=row, col=col)

    fig.update_yaxes(autorange='reversed', row=row, col=col)

fig.update_layout(
    height=650,
    width=1000,
    title_text=f"Top 5 Pemain Muda (≤ {age_threshold:.1f} Tahun) Berdasarkan G+A per Liga",
    showlegend=False,
    title_x=0.5
)

fig.show()

In [77]:
# Membuat leaderboard interaktif untuk 30 pemain muda teratas berdasarkan G+A.
young_players['G+A'] = young_players['Gls'] + young_players['Ast']

young_leaderboard = young_players.sort_values('G+A', ascending=False)

fig = px.bar(
    young_leaderboard.head(30),
    x='G+A',
    y='Player',
    color='G+A',
    color_continuous_scale='viridis',
    orientation='h',
    text='G+A',
    hover_data={
        'Player': True,
        'Gls': True,
        'Ast': True,
        'G+A': True,
        'Age': True,
        'Squad': True,
        'Comp': True
    },
    title=f"Leaderboard Pemain Muda (≤ {age_threshold:.1f} Tahun) Berdasarkan G+A"
)

fig.update_layout(
    yaxis=dict(autorange='reversed'),
    xaxis_title="Gol + Assist",
    yaxis_title="Pemain",
    title_x=0.5,
    height=800
)

fig.show()

**1. Bradley Barcola dan Florian Wirtz adalah Pemain Muda Paling Berpengaruh dalam Kontribusi Gol & Assist di Eropa:**
* **Grafik "Leaderboard Pemain Muda ($\le$ 21.0 Tahun) Berdasarkan G+A" dengan jelas menyoroti Bradley Barcola (24 G+A) dan Florian Wirtz (22 G+A) sebagai pemain muda dengan kontribusi ofensif gabungan tertinggi.** Angka ini sangat impresif dan menunjukkan bahwa mereka adalah pemain kunci yang secara langsung berkontribusi pada gol tim mereka.

**2. Bakat Muda Paling Kreatif (Assist) Berasal dari Berbagai Liga:**
* **Grafik "Top 5 Pencetak Assist Pemain Muda ($\le$ 21.0 Tahun) per Liga" menunjukkan Lamine Yamal (La Liga) sebagai pencetak *assist* muda terbanyak dengan 13 *assist*.**
* **Florian Wirtz (Bundesliga) juga menonjol sebagai pencetak *assist* dengan 12 *assist*,** yang menjelaskan mengapa ia memiliki G+A yang sangat tinggi.
* Rayan Cherki (Ligue 1) juga merupakan *playmaker* yang sangat produktif dengan 11 *assist*.
* Ini menunjukkan bahwa ada bakat-bakat kreatif yang tersebar di berbagai liga, meskipun La Liga, Bundesliga, dan Ligue 1 memiliki *playmaker* muda paling menonjol.

**3. Konsentrasi Bakat G+A Tinggi di Ligue 1, La Liga, dan Bundesliga:**

* **Grafik "Top 5 Pemain Muda ($\le$ 21.0 Tahun) Berdasarkan G+A per Liga" menegaskan bahwa Ligue 1, La Liga, dan Bundesliga adalah liga yang paling banyak menghasilkan pemain muda dengan kontribusi G+A yang sangat tinggi.**
    * **Ligue 1:** Bradley Barcola (24 G+A) dan Rayan Cherki (19 G+A) memimpin.
    * **La Liga:** Lamine Yamal (22 G+A) dan Jude Bellingham (17 G+A) menunjukkan pengaruh besar.
    * **Bundesliga:** Florian Wirtz (22 G+A) dan Benjamin Šeško (18 G+A) adalah bintang muda yang bersinar.
* Ini sejalan dengan *insight* sebelumnya bahwa Ligue 1 memiliki banyak pemain muda dan Bundesliga memiliki pencetak gol muda yang produktif.

**4. Peran All-Rounder Jude Bellingham dan Xavi Simons:**
* Jude Bellingham dan Xavi Simons, yang sebelumnya muncul dalam statistik menit bermain dan rasio *starter*, kini juga menunjukkan kontribusi G+A yang sangat baik (17 G+A dan 16 G+A secara berturut-turut). Ini mengukuhkan mereka sebagai pemain *all-rounder* yang tidak hanya dominan dalam aspek *playmaking* dan kontrol lini tengah, tetapi juga efektif dalam mencetak gol dan *assist*.

**5. Bukti Dampak Langsung Pemain Muda pada Performa Tim:**
* Angka G+A yang tinggi ini adalah bukti nyata bahwa pemain muda tidak hanya menjadi pelengkap, tetapi secara langsung memengaruhi hasil pertandingan. Mereka adalah penentu dan pencipta momen-momen penting dalam serangan tim.
* Keberadaan mereka di puncak daftar G+A menunjukkan bahwa investasi pada pemain muda yang tepat dapat memberikan dividen instan dalam hal output ofensif.

**6. Perbandingan dengan Grafik Sebelumnya (Kontribusi Gol Saja):**
* Sebelumnya kita melihat bahwa kontribusi *assist* pemain muda (15.6%) sedikit lebih tinggi dari gol (14%). Grafik ini memberikan detail siapa saja pemain yang mendorong angka-angka tersebut, dan menegaskan peran penting *playmaker* muda.
* Beberapa nama yang muncul di top *scorer* gol muda juga muncul di top G+A (misalnya Emanuel Emegha, Benjamin Šeško), menunjukkan bahwa mereka adalah penyerang yang komplit.

Secara keseluruhan, analisis ini menyoroti bahwa ada beberapa talenta muda yang luar biasa di sepak bola Eropa yang tidak hanya sering bermain, tetapi juga sangat efektif dalam menghasilkan gol dan *assist* untuk tim mereka. Mereka adalah bintang masa depan yang sudah memberikan dampak signifikan di masa sekarang.

## ♟ Analisis Posisi Pemain Muda
Bagian ini menganalisis distribusi posisi bermain pemain muda dan rata-rata menit bermain yang mereka dapatkan di setiap posisi.

In [78]:
# Menghitung jumlah pemain muda berdasarkan posisi bermain.
young_pos_counts = young_players['Pos'].value_counts().reset_index()
young_pos_counts.columns = ['Posisi', 'Jumlah Pemain']

print(young_pos_counts)

# Membuat bar chart untuk jumlah pemain muda berdasarkan posisi.
fig = px.bar(
    young_pos_counts,
    x='Jumlah Pemain',
    y='Posisi',
    orientation='h',
    text='Jumlah Pemain',
    color='Jumlah Pemain',
    color_continuous_scale='matter',
    title='Jumlah Pemain Muda Berdasarkan Posisi'
)

fig.update_layout(
    yaxis=dict(categoryorder='total ascending'), # Mengurutkan posisi berdasarkan jumlah pemain
    title_x=0.5
)

fig.show()

  Posisi  Jumlah Pemain
0     DF            185
1     MF            159
2     FW             96
3  FW,MF             84
4  MF,FW             74
5  DF,MF             30
6  MF,DF             27
7  DF,FW             15
8     GK             14
9  FW,DF              8


In [79]:
# Menghitung rata-rata menit bermain pemain muda per posisi.
avg_min_per_pos = young_players.groupby('Pos')['Min'].mean().round(2).sort_values(ascending=False).reset_index()
avg_min_per_pos.columns = ['Posisi', 'Rata-rata Menit']
print(avg_min_per_pos)

# Membuat bar chart untuk rata-rata menit bermain pemain muda per posisi.
fig = px.bar(
    avg_min_per_pos,
    x='Rata-rata Menit',
    y='Posisi',
    orientation='h',
    text='Rata-rata Menit',
    color='Rata-rata Menit',
    color_continuous_scale='viridis',
    title='Rata-rata Menit Bermain Pemain Muda per Posisi'
)

fig.update_layout(
    yaxis=dict(categoryorder='total ascending'),
    title_x=0.5
)

fig.show()

  Posisi  Rata-rata Menit
0     GK           885.86
1  DF,MF           791.57
2  MF,DF           784.93
3  MF,FW           752.74
4     MF           744.36
5     DF           742.39
6  FW,MF           647.99
7  DF,FW           509.80
8     FW           472.33
9  FW,DF            50.00


In [80]:
# Membandingkan rata-rata menit bermain dan jumlah pemain muda per posisi
avg_minutes = young_players.groupby('Pos')['Min'].mean().round(2)
count_players = young_players['Pos'].value_counts().sort_index()

pos_stats = pd.DataFrame({
    'Rata-rata Menit': avg_minutes,
    'Jumlah Pemain': count_players
}).reset_index().rename(columns={'index': 'Pos'})
print(pos_stats)

fig = go.Figure()

# Bar chart untuk rata-rata menit bermain.
fig.add_trace(go.Bar(
    x=pos_stats['Pos'],
    y=pos_stats['Rata-rata Menit'],
    name='Rata-rata Menit',
    marker_color='#C95792',
    yaxis='y1'
))

# Bar chart untuk jumlah pemain.
fig.add_trace(go.Bar(
    x=pos_stats['Pos'],
    y=pos_stats['Jumlah Pemain'],
    name='Jumlah Pemain',
    marker_color='#F8B55F',
    yaxis='y2'
))

# Mengatur tata letak dengan dua sumbu Y untuk perbandingan yang lebih baik.
fig.update_layout(
    title='Rata-rata Menit & Jumlah Pemain Muda per Posisi',
    xaxis_title='Posisi',
    yaxis=dict(
        title='Rata-rata Menit',
        titlefont=dict(color='#3D365C'),
        tickfont=dict(color='#3D365C'),
        side='left'
    ),
    yaxis2=dict(
        title='Jumlah Pemain',
        titlefont=dict(color='#3D365C'),
        tickfont=dict(color='#3D365C'),
        overlaying='y',
        side='right'
    ),
    barmode='group',
    title_x=0.5,
    legend=dict(x=0.5, xanchor='center', y=1.1, orientation='h')
)

fig.show()

     Pos  Rata-rata Menit  Jumlah Pemain
0     DF           742.39            185
1  DF,FW           509.80             15
2  DF,MF           791.57             30
3     FW           472.33             96
4  FW,DF            50.00              8
5  FW,MF           647.99             84
6     GK           885.86             14
7     MF           744.36            159
8  MF,DF           784.93             27
9  MF,FW           752.74             74


**1. Penjaga Gawang (GK) Mendapatkan Rata-rata Menit Bermain Tertinggi, Namun Jumlahnya Paling Sedikit:**

* **Grafik "Rata-rata Menit Bermain Pemain Muda per Posisi" dan "Rata-rata Menit & Jumlah Pemain Muda per Posisi" dengan jelas menunjukkan bahwa posisi Penjaga Gawang (GK) memiliki rata-rata menit bermain pemain muda tertinggi (sekitar 885.86 menit).**
* Namun, **grafik "Jumlah Pemain Muda Berdasarkan Posisi" menunjukkan bahwa GK adalah posisi dengan jumlah pemain muda paling sedikit (hanya 14 pemain).**
* Ini menyiratkan bahwa meskipun sangat jarang ada pemain muda yang menjadi penjaga gawang utama, ketika ada, mereka cenderung mendapatkan kepercayaan penuh dan banyak menit bermain. Posisi GK membutuhkan konsistensi dan kematangan, sehingga klub tidak akan mengambil risiko dengan GK muda jika mereka tidak benar-benar percaya pada kemampuannya.

**2. Pemain Bertahan (DF) dan Gelandang (MF) Memiliki Jumlah Pemain Muda Terbanyak, Namun Menit Bermainnya Beragam:**

* **Grafik "Jumlah Pemain Muda Berdasarkan Posisi" (newplot (21).png) menunjukkan bahwa DF (Pemain Bertahan) dan MF (Gelandang) adalah posisi dengan jumlah pemain muda terbanyak, masing-masing 185 dan 159 pemain.**
* Untuk menit bermain:
    * **DF dan MF memiliki rata-rata menit bermain yang cukup tinggi (sekitar 742-744 menit),** menunjukkan bahwa ada banyak pemain muda di posisi ini yang mendapatkan waktu bermain reguler.
    * Posisi "DF,MF" (kemungkinan pemain yang bisa bermain sebagai bek dan gelandang, seperti bek sayap atau gelandang bertahan) dan "MF,DF" juga menunjukkan rata-rata menit bermain yang tinggi (sekitar 791-784 menit), menunjukkan fleksibilitas posisi ini dihargai dalam konteks pemain muda.

**3. Penyerang (FW) dan Fleksibilitas Posisi Ofensif Memiliki Menit Bermain yang Lebih Rendah, Meskipun Jumlahnya Cukup Banyak:**

* **Posisi FW (Penyerang) memiliki rata-rata menit bermain pemain muda terendah di antara posisi inti (sekitar 472.33 menit),** meskipun jumlah pemainnya cukup banyak (96 pemain).
* Demikian pula, posisi "FW,DF" (pemain penyerang yang juga bisa bermain sebagai bek, mungkin *wing-back* ofensif) memiliki rata-rata menit bermain yang sangat rendah (50 menit) dan jumlah pemain yang sedikit (8 pemain).
* Posisi "FW,MF" (penyerang yang juga bisa bermain sebagai gelandang serang/sayap) memiliki jumlah pemain muda yang signifikan (84 pemain) tetapi rata-rata menit bermain yang lebih rendah (sekitar 647.99 menit) dibandingkan DF/MF.
* Ini menunjukkan bahwa meskipun banyak pemain muda mencoba posisi penyerang, persaingan di lini depan mungkin lebih ketat, atau klub lebih memilih penyerang yang lebih berpengalaman. Pemain muda di posisi ofensif mungkin lebih sering masuk sebagai pemain pengganti.

**4. Pentingnya Fleksibilitas Posisi dalam Mendapatkan Menit Bermain:**

* Posisi yang menunjukkan fleksibilitas (seperti DF,MF; MF,DF; MF,FW; FW,MF) umumnya memiliki rata-rata menit bermain yang lebih tinggi dibandingkan posisi tunggal seperti FW.
* Ini menunjukkan bahwa pemain muda yang memiliki kemampuan untuk bermain di lebih dari satu posisi (terutama antara lini tengah dan belakang) lebih mungkin untuk mendapatkan menit bermain yang signifikan. Fleksibilitas meningkatkan peluang mereka untuk masuk ke dalam tim.

**5. Distribusi Pemain Muda di Tim:**

* Mayoritas pemain muda terkonsentrasi di lini belakang (DF) dan tengah (MF). Hal ini bisa jadi karena posisi ini seringkali membutuhkan energi dan agresivitas, yang cocok dengan atribut pemain muda, atau karena ada lebih banyak ruang untuk rotasi dan pengembangan di posisi ini.

**Kesimpulan Gabungan:**

Secara keseluruhan, analisis ini menunjukkan bahwa:

* **Penjaga gawang muda yang berhasil menembus tim utama sangat dihargai dan mendapatkan banyak menit bermain, meskipun jumlah mereka sedikit.**
* **Pemain bertahan dan gelandang muda adalah kelompok terbesar dan secara konsisten mendapatkan menit bermain yang layak.**
* **Pemain muda dengan kemampuan bermain di beberapa posisi (fleksibilitas) cenderung mendapatkan lebih banyak kesempatan bermain.**
* **Penyerang muda menghadapi persaingan yang lebih ketat untuk mendapatkan menit bermain reguler, meskipun ada banyak talenta di posisi ini.**


## 🧿 Efisiensi Pemain Muda (Gol vs xG & Assist vs xAG)
Bagian ini mengukur seberapa efisien seorang pemain dalam menyelesaikan peluang (untuk pencetak gol) atau menciptakan peluang berkualitas (untuk playmaker) menggunakan metrik Expected Goals (xG) dan Expected Assisted Goals (xAG).


In [81]:
# 1. Filter pemain muda yang mencetak minimal 10 gol, dan urutkan berdasarkan gol per 90 menit.
young_players = df_cleaned[(df_cleaned['Age'] <= age_threshold) & (df_cleaned['Gls'] >= 10)].sort_values(['Gls_90', 'Gls'], ascending=False)
data = young_players[['Player', 'Gls', 'Age', 'Squad', 'Gls_90', 'Min']].head(10).reset_index(drop=True)

# 2. Membuat bar chart untuk 10 pencetak gol muda paling efektif (berdasarkan Gls_90).
fig = px.bar(
    data,
    x='Player',
    y='Gls_90',
    color='Gls_90',
    color_continuous_scale='viridis',
    hover_data=['Age', 'Squad', 'Min', 'Gls'],
    labels={'Gls_90': 'Gol per 90 Menit'},
    height=400,
    title='Top 10 Pencetak Gol Paling Efektif (Pemain Muda)',
    text='Gls_90'
)

fig.update_traces(
    texttemplate='%{text:.2f}', # Menampilkan 2 angka di belakang koma pada label teks
    textposition='outside'
)

fig.update_layout(
    title_x=0.5,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis_title='Gol per 90 Menit'
)

fig.show()

In [82]:
# 1. Filter pemain muda yang mencetak minimal 10 gol dan hitung rasio konversi Gol/xG.
young_players = df_cleaned[(df_cleaned['Age'] <= age_threshold) & (df_cleaned['Gls'] >= 10)]
data = young_players[['Player', 'Gls', 'Age', 'Squad', 'xG']].reset_index(drop=True)
data['conv_Gls_xG'] = data['Gls'] / data['xG'] # Rasio konversi Gol dibagi Expected Goals
data = data.sort_values('conv_Gls_xG', ascending=False).head(10)

# 2. Membuat plot kombinasi bar dan garis untuk Gol, xG, dan rasio konversi Gol/xG.
fig = go.Figure()

# Bar chart untuk Gol aktual.
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['Gls'],
    name='Gol',
    marker_color='#EF9651',
    yaxis='y'
))

# Bar chart untuk Expected Goals (xG).
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['xG'],
    name='xG',
    marker_color='#EC5228',
    yaxis='y'
))

# Garis untuk rasio konversi Gol/xG.
fig.add_trace(go.Scatter(
    x=data['Player'],
    y=data['conv_Gls_xG'],
    name='Tingkat Konversi (Gol/xG)',
    marker_color='#3F7D58',
    mode='lines+markers+text',
    text=[f'{val:.2f}' for val in data['conv_Gls_xG']],
    textposition='top center',
    yaxis='y2' # Menggunakan sumbu Y kedua
))

fig.update_layout(
    title='Gol vs xG dan Tingkat Konversi Pemain Muda',
    xaxis=dict(title='Pemain'),
    yaxis=dict(
        title='Jumlah Gol & xG',
        titlefont=dict(color='#440154'),
        tickfont=dict(color='#440154')
    ),
    yaxis2=dict(
        title='Tingkat Konversi (Gol/xG)',
        titlefont=dict(color='#440154'),
        tickfont=dict(color='#440154'),
        overlaying='y', # Menumpuk sumbu Y kedua di atas yang pertama
        side='right'
    ),
    barmode='group',
    title_x=0.5,
    legend=dict(x=0.5, xanchor='center', y=1.15, orientation='h'),
    height=500
)

fig.show()


In [83]:
# 1. Filter pemain muda dengan assist di kuartil 95% ke atas, urutkan berdasarkan assist per 90 menit.
young_players = df_cleaned[(df_cleaned['Age'] <= age_threshold) & (df_cleaned['Ast'] >= df_cleaned['Ast'].quantile(0.95))].sort_values(['Ast_90', 'Ast'], ascending=False)
data = young_players[['Player', 'Ast', 'Age', 'Squad', 'Ast_90', 'Min']].head(10).reset_index(drop=True)

# Membuat bar chart untuk 10 pemain muda dengan assist per 90 menit tertinggi.
fig = px.bar(
    data,
    x='Player',
    y='Ast_90',
    hover_data=['Age', 'Squad', 'Min', 'Ast'],
    color='Ast_90',
    color_continuous_scale='magma',
    labels={'Ast_90': 'Assist per 90 Menit'},
    height=400,
    title='Top 10 Pemain Muda dengan Assist per 90 Menit Tertinggi',
    text='Ast_90'
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.update_layout(
    title_x=0.5,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis_title='Assist per 90 Menit'
)

fig.show()


In [84]:
# 1. Filter pemain muda dengan assist di kuartil 95% ke atas dan hitung rasio konversi Assist/xAG.
young_players = df_cleaned[(df_cleaned['Age'] <= age_threshold) & (df_cleaned['Ast'] >= df_cleaned['Ast'].quantile(0.95))]
data = young_players[['Player', 'Ast', 'Age', 'Squad', 'xAG']].reset_index(drop=True)
data['conv_Ast_xAG'] = data['Ast'] / data['xAG'] # Rasio konversi Assist dibagi Expected Assisted Goals
data = data.sort_values('conv_Ast_xAG', ascending=False).head(10)

# 2. Membuat plot kombinasi bar dan garis untuk Assist, xAG, dan rasio konversi Assist/xAG.
fig = go.Figure()

# Bar chart untuk Assist aktual.
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['Ast'],
    name='Assist',
    marker_color='#0d0887',
    yaxis='y'
))

# Bar chart untuk Expected Assisted Goals (xAG).
fig.add_trace(go.Bar(
    x=data['Player'],
    y=data['xAG'],
    name='xAG',
    marker_color='#cc4678',
    yaxis='y'
))

# Garis untuk rasio konversi Assist/xAG.
fig.add_trace(go.Scatter(
    x=data['Player'],
    y=data['conv_Ast_xAG'],
    name='Tingkat Konversi (Assist/xAG)',
    marker_color='#f0f951',
    mode='lines+markers+text',
    text=[f'{val:.2f}' for val in data['conv_Ast_xAG']],
    textposition='top center',
    yaxis='y2'
))

fig.update_layout(
    title='Assist vs xAG dan Tingkat Konversi Pemain Muda',
    xaxis=dict(title='Pemain'),
    yaxis=dict(
        title='Jumlah Assist & xAG',
        titlefont=dict(color='#0d0887'),
        tickfont=dict(color='#0d0887')
    ),
    yaxis2=dict(
        title='Tingkat Konversi (Assist/xAG)',
        titlefont=dict(color='#0d0887'),
        tickfont=dict(color='#0d0887'),
        overlaying='y',
        side='right'
    ),
    barmode='group',
    title_x=0.5,
    legend=dict(x=0.5, xanchor='center', y=1.15, orientation='h'),
    height=500
)

fig.show()

**1. Mika Biereth adalah Pencetak Gol Muda Paling Efektif per 90 Menit:**

* **Grafik "Top 10 Pencetak Gol Paling Efektif (Pemain Muda)" menyoroti Mika Biereth dengan rasio gol per 90 menit tertinggi (0.93).** Angka ini sangat luar biasa, menunjukkan bahwa ia sangat efisien dalam mencetak gol ketika ia berada di lapangan.
* Pemain lain seperti Jamal Musiala (0.60), Bradley Barcola (0.58), dan Lucas Stassin (0.57) juga menunjukkan efektivitas gol yang sangat tinggi.
* Ini adalah metrik penting untuk mengidentifikasi "insting pembunuh" seorang penyerang. Pemain dengan G/90 yang tinggi berarti mereka memaksimalkan peluang mereka di waktu bermain yang diberikan.

**2. Rayan Cherki Memimpin dalam Efektivitas Assist per 90 Menit:**

* **Grafik "Top 10 Pemain Muda dengan Assist per 90 Menit Tertinggi" menunjukkan Rayan Cherki (0.48) sebagai pemain muda dengan rasio *assist* per 90 menit tertinggi.** Ini mengindikasikan bahwa ia adalah *playmaker* yang sangat kreatif dan efektif dalam menciptakan peluang bagi rekan satu timnya.
* Florian Wirtz (0.46) dan Lamine Yamal (0.41) juga menunjukkan efektivitas *assist* yang sangat tinggi, menguatkan status mereka sebagai kreator peluang utama.
* Metrik ini membantu mengidentifikasi *playmaker* yang paling efisien dalam menciptakan *assist*, tidak hanya berdasarkan total *assist*, tetapi juga efektivitasnya per waktu bermain.

**3. Tingkat Konversi Gol (Gol vs xG) Menyoroti Kemampuan Finishing:**

* **Grafik "Gol vs xG dan Tingkat Konversi Pemain Muda" memberikan wawasan tentang kualitas *finishing* pemain muda.**
* **Andrey Santos dan Xavi Simons menunjukkan tingkat konversi (Gol/xG) yang sangat tinggi (1.96).** Ini berarti mereka mencetak hampir dua kali lipat gol dari yang diharapkan berdasarkan kualitas peluang yang mereka dapatkan. Ini adalah indikasi *finishing* klinis dan kemampuan untuk mengubah peluang sulit menjadi gol.
* Lucas Stassin (1.72) dan Paul Nebel (1.71) juga memiliki tingkat konversi yang sangat baik.
* Sebaliknya, pemain seperti Florian Wirtz dan Bradley Barcola memiliki rasio konversi mendekati 1.00 (1.04 dan 1.06), yang berarti mereka mencetak gol sesuai dengan kualitas peluang yang diterima, menunjukkan *finishing* yang solid dan efisien.
* Tingkat konversi Gol/xG yang tinggi menunjukkan pemain yang "overperform" ekspektasi, seringkali karena *finishing* yang luar biasa atau kemampuan untuk mencetak gol dari situasi yang sulit.

**4. Tingkat Konversi Assist (Assist vs xAG) Menyoroti Visi dan Kualitas Umpan Kunci:**

* **Grafik "Assist vs xAG dan Tingkat Konversi Pemain Muda"mirip dengan Gol/xG, tetapi untuk *assist*.**
* **Benjamin Šeško memiliki tingkat konversi *assist* (Assist/xAG) tertinggi (2.38).** Ini menunjukkan bahwa umpan-umpan kuncinya seringkali berubah menjadi *assist* melebihi ekspektasi. Ini bisa jadi karena ia memberikan umpan yang sangat presisi atau karena rekan setimnya memiliki *finishing* yang sangat baik dari umpan-umpannya.
* Christantus Uche (2.07) dan Jude Bellingham (2.05) juga memiliki rasio konversi *assist* yang sangat tinggi, menunjukkan kualitas umpan dan visi mereka dalam menciptakan peluang.
* Tingkat konversi Assist/xAG yang tinggi menunjukkan pemain yang efisien dalam menciptakan *assist*, baik karena umpan kuncinya yang berkualitas tinggi atau kemampuan rekan setimnya untuk mengkonversi peluang dari umpan tersebut.

**Kesimpulan Akhir dari Seluruh Analisis:**

Dari seluruh grafik yang telah kita analisis, berikut adalah gambaran komprehensif mengenai potensi pemain muda di top 5 liga:

1.  **Ligue 1** adalah "pabrik" pemain muda, memiliki jumlah pemain muda terbanyak dan proporsi klub yang tinggi. Ini juga menghasilkan pencetak gol dan *assist* muda yang sangat produktif (misalnya Emanuel Emegha, Bradley Barcola, Rayan Cherki).
2.  **La Liga** adalah liga terbaik dalam memberikan peran *starter* kepada pemain muda secara konsisten. Mereka juga memiliki talenta ofensif yang luar biasa (misalnya Lamine Yamal, Jude Bellingham) yang menunjukkan efektivitas gol dan *assist* yang tinggi.
3.  **Premier League** memberikan menit bermain rata-rata tertinggi bagi pemain muda, menunjukkan lingkungan yang kompetitif namun juga memberikan kepercayaan pada talenta muda. Mereka juga memiliki beberapa pencetak gol efektif per 90 menit (misalnya Liam Delap).
4.  **Bundesliga** adalah rumah bagi beberapa pencetak gol dan kreator yang sangat efektif (misalnya Benjamin Šeško, Florian Wirtz, Jamal Musiala, Xavi Simons), menunjukkan kualitas individu yang tinggi dalam produksi ofensif.
5.  **Serie A** secara umum memiliki rata-rata menit bermain dan rasio *starter* yang lebih rendah untuk pemain muda dibandingkan liga lain, dan kontribusi ofensifnya juga lebih rendah. Namun, ada beberapa individu yang menonjol seperti Mika Biereth dengan efektivitas gol yang luar biasa per 90 menit.

**Pemain Kunci yang Menonjol di Berbagai Metrik:**

* **Milos Kerkez & Zion Suzuki:** Konsisten sebagai pemain inti, menit bermain tinggi, dan rasio *starter* sempurna. Menunjukkan keandalan dan kepercayaan dari pelatih.
* **Bradley Barcola & Florian Wirtz:** Pemimpin mutlak dalam kontribusi G+A, menunjukkan dampak ofensif langsung yang luar biasa. Keduanya juga sangat efektif per 90 menit dalam *assist* atau gol.
* **Rayan Cherki & Lamine Yamal:** *Playmaker* muda yang sangat efektif dalam menciptakan *assist*, terutama per 90 menit.
* **Mika Biereth:** Pencetak gol paling efektif per 90 menit, menunjukkan insting gol yang luar biasa.
* **Andrey Santos & Xavi Simons:** Menunjukkan *finishing* yang sangat klinis dengan tingkat konversi Gol/xG yang sangat tinggi.
* **Jude Bellingham:** Pemain *all-rounder* yang menonjol dalam menit bermain, G+A, dan tingkat konversi *assist*.

Secara keseluruhan, data ini memberikan gambaran yang kaya tentang lanskap pemain muda di sepak bola Eropa, menyoroti liga-liga yang paling berinvestasi pada talenta muda, klub-klub yang paling aktif dalam pengembangan mereka, dan individu-individu yang sudah menunjukkan potensi luar biasa dalam berbagai aspek permainan.

## 🕹 Kontribusi Pemain Muda: Umpan Progresif
Bagian ini menganalisis kontribusi pemain muda dalam melakukan umpan progresif (PrgP) dan hubungannya dengan Expected Assisted Goals (xAG), yang menunjukkan kemampuan mereka dalam memajukan bola dan menciptakan peluang.


In [85]:
# 1. Filter pemain muda dan ambil 10 teratas berdasarkan umpan progresif (PrgP).
top_passes_progressive = young_players[['Player', 'Squad', 'Comp', 'Age','PrgP', 'xAG', 'Ast']].sort_values('PrgP', ascending=False).head(10).reset_index()

# Membuat bar chart untuk 10 pemain muda dengan umpan progresif terbanyak.
px.bar(
    top_passes_progressive,
    x='Player',
    y='PrgP',
    hover_data=['Squad', 'Age', 'Ast', 'PrgP', 'xAG'],
    title='Top 10 Pemain Muda dengan Umpan Progresif (PrgP)',
    color='PrgP',
    color_continuous_scale='tropic'
).update_layout(title_x=0.5).show()

In [96]:
# Menghitung total umpan progresif dan Expected Assisted Goals (PrgP_xAG).
young_players['PrgP_xAG'] = young_players['PrgP'] + young_players['xAG']
top_progressors = young_players.sort_values('PrgP_xAG', ascending=False).head(10)

# Membuat bar chart untuk 10 pemain muda teratas berdasarkan total PrgP + xAG.
px.bar(
    top_progressors,
    x='Player',
    y='PrgP_xAG',
    hover_data=['Squad', 'Age', 'Ast', 'PrgP', 'xAG'],
    title='Top 10 Pemain Muda (Umpan Progresif + xAG)',
    color='PrgP_xAG',
    color_continuous_scale='Plasma'
).update_layout(title_x=0.5).show()

In [97]:
# Membuat scatter plot untuk menunjukkan hubungan antara umpan progresif (PrgP) dan xAG,
# dengan ukuran dan warna titik berdasarkan jumlah assist (Ast).
top_progressors_players = young_players.sort_values('PrgP_xAG', ascending=False).head(10)['Player'].tolist()
fig = px.scatter(
    young_players, 
    x='PrgP', 
    y='xAG', 
    color='Ast', 
    size='Ast', 
    hover_data=['Player', 'Squad', 'Age'],
    title='Hubungan Umpan Progresif dan xAG (Ukuran dan Warna: Assist)',
    color_continuous_scale='Viridis'
)

# Menambahkan anotasi untuk setiap pemain
for index, row in young_players.iterrows():
    if row['Player'] in top_progressors_players:
        fig.add_annotation(
            x=row['PrgP'],
            y=row['xAG'],
            text=row['Player'],
            showarrow=False, # Tidak menampilkan panah
            yshift=10, # Menggeser teks sedikit ke atas dari titik
            xshift=0, # Menggeser teks sedikit ke kanan atau kiri
            font=dict(size=8, color="black") # Ukuran dan warna font
        )

fig.update_layout(title_x=0.5)
fig.show()

## 📕Ringkasan 


**Analisis Komprehensif dan Insight Mengenai Potensi Pemain Muda di Top 5 Liga:**

**I. Volume dan Komitmen Liga terhadap Pemain Muda:**

1.  **Ligue 1 (Prancis): Produsen Bakat Terbesar & Inkubator Klub:**
    * **Jumlah Pemain Muda Terbanyak:** Grafik "Jumlah Pemain Muda per Liga (< 21 Tahun)" (newplot (2).png) menunjukkan Ligue 1 memiliki konsentrasi pemain muda paling tinggi (sekitar 180 pemain). Ini mengindikasikan liga ini berfungsi sebagai wadah utama untuk pengembangan talenta muda.
    * **Klub yang Investif:** Grafik "Top 5 Klub dengan Jumlah Pemain Muda Terbanyak" (newplot (3).png) dan "Top 10 Klub dengan Proporsi Pemain Muda Tertinggi" (newplot (4).png) menegaskan bahwa klub-klub Ligue 1 seperti Strasbourg dan Reims memiliki jumlah dan proporsi pemain muda yang sangat tinggi (lebih dari 50%). Ini menunjukkan filosofi klub yang kuat dalam mengandalkan dan mengembangkan pemain muda.

2.  **La Liga (Spanyol): Liga Starter dan Kreator Utama:**
    * **Rasio Starter Tertinggi:** Grafik "Tingkat Starter Pemain Muda per Liga" (newplot (11).png) secara konsisten menunjukkan La Liga memiliki rasio *starter* pemain muda tertinggi (sekitar 0.60 atau 60%). Ini adalah bukti bahwa pemain muda di La Liga tidak hanya banyak, tetapi juga dipercaya untuk memulai pertandingan.
    * **Talenta Kreatif Dominan:** Kehadiran Pedri yang sangat dominan dalam "Umpan Progresif (PrgP)" (newplot (28).png) dan "Umpan Progresif + xAG" (newplot (29).png), serta Lamine Yamal sebagai top *assist* muda di La Liga (newplot (16).png), menunjukkan bahwa liga ini menghasilkan *playmaker* muda kelas dunia yang sangat berpengaruh dalam membangun serangan. Barcelona dan Real Madrid (newplot (4).png) juga memiliki proporsi pemain muda yang tinggi, mengukuhkan komitmen mereka.

3.  **Premier League (Inggris): Menit Bermain yang Kompetitif:**
    * **Rata-rata Menit Bermain Tertinggi:** Meskipun jumlah pemain muda keseluruhan tidak sebanyak Ligue 1, Premier League memberikan rata-rata menit bermain tertinggi bagi pemain muda. Ini menunjukkan bahwa pemain muda yang berhasil menembus tim di Premier League mendapatkan pengalaman lapangan yang substansial.
    * **Klub Pendukung Pemain Muda:** Ipswich Town, Celta Vigo (meskipun La Liga, ia di top klub menit bermain), Nott'ham Forest, Newcastle Utd, dan Brighton (newplot (5).png) menunjukkan bahwa ada klub-klub di Premier League yang proaktif dalam memberikan menit bermain signifikan kepada pemain muda.

4.  **Bundesliga (Jerman): Efisiensi Ofensif dan Bakat yang Fokus:**
    * **Produktivitas Gol & Assist Tinggi:** Bundesliga memiliki pencetak gol muda yang sangat produktif (Benjamin Šeško, Jamal Musiala - newplot (15).png) dan kreator *assist* yang efektif (Florian Wirtz - newplot (16).png). Pemain seperti Xavi Simons juga menunjukkan kontribusi G+A yang tinggi (newplot (17).png, newplot (18).png).
    * **Efisiensi Konversi:** Benjamin Šeško menonjol dengan tingkat konversi *assist* (Assist/xAG) tertinggi (newplot (22).png), menunjukkan kualitas umpan kuncinya.

5.  **Serie A (Italia): Tantangan Menit Bermain, Tapi Ada Potensi Efektivitas:**
    * **Menit Bermain & Rasio Starter Terendah:** Serie A secara keseluruhan menunjukkan angka terendah dalam rata-rata menit bermain dan rasio *starter* pemain muda. Ini mungkin menunjukkan liga yang lebih mengutamakan pengalaman atau taktik yang ketat.
    * **Efektivitas Individu:** Meskipun demikian, ada mutiara tersembunyi seperti Mika Biereth yang merupakan pencetak gol paling efektif per 90 menit (newplot (25).png), menunjukkan bahwa efisiensi individu bisa sangat tinggi meskipun menit bermain terbatas.

**II. Peran dan Kontribusi Pemain Muda dalam Permainan:**

1.  **Dampak Ofensif Langsung yang Signifikan:**
    * **Kontribusi yang Terukur:** Pemain muda menyumbang 14% dari total gol, 15.6% dari total *assist*, dan 14.7% dari total G+A (newplot (13).png). Angka ini sangat berarti mengingat jumlah mereka dibandingkan seluruh pemain di liga.
    * **Kecenderungan untuk Meng-Assist:** Kontribusi *assist* yang sedikit lebih tinggi dari gol menunjukkan bahwa banyak pemain muda yang berposisi sebagai kreator peluang.

2.  **Pemain Muda sebagai Pilar Utama (Bukan Hanya Pelapis):**
    * **Menit Bermain & Starter Konsisten:** Milos Kerkez menonjol sebagai pemain muda dengan menit bermain dan jumlah pertandingan terbanyak (newplot (7).png, newplot (9).png). Bersama Cristhian Mosquera, Zion Suzuki, Bart Verbruggen, Omar El Hilali, Levi Colwill, dan lainnya, mereka menunjukkan rasio *starter* 1.00 dalam minimal 10 hingga 20 pertandingan (newplot (10).png, newplot (12).png). Ini adalah indikasi kuat bahwa mereka bukan hanya talenta, tetapi sudah menjadi pemain inti yang sangat diandalkan.

3.  **Keberagaman Posisi dan Peran Kunci:**
    * **Penjaga Gawang (GK): Tinggi dalam Menit Bermain, Rendah dalam Jumlah:** Meskipun jumlah GK muda sangat sedikit, mereka yang berhasil menembus mendapatkan menit bermain rata-rata tertinggi (newplot (20).png, newplot (21).png). Ini menunjukkan kepercayaan mutlak pada bakat-bakat langka ini.
    * **Bek (DF) & Gelandang (MF): Jumlah Terbanyak & Menit Bermain Reguler:** Posisi ini menjadi inkubator utama pemain muda, mendapatkan banyak menit bermain reguler, menunjukkan bahwa mereka adalah fondasi bagi tim.
    * **Fleksibilitas Posisi:** Pemain muda dengan kemampuan di beberapa posisi (misalnya DF,MF) cenderung mendapatkan menit bermain yang lebih tinggi, menggarisbawahi pentingnya adaptabilitas.

4.  **Efisiensi dalam Produksi Ofensif (Per 90 Menit):**
    * **Pencetak Gol Efektif:** Mika Biereth (0.93 gol/90) adalah contoh utama pemain muda yang sangat efisien dalam mencetak gol di waktu bermain yang diberikan (newplot (25).png).
    * **Kreator Assist Efektif:** Rayan Cherki (0.48 assist/90) dan Florian Wirtz (0.46 assist/90) adalah contoh *playmaker* muda yang sangat efektif dalam menciptakan peluang (newplot (23).png).

5.  **Kemampuan *Finishing* dan Visi Umpan Kunci yang Superior:**
    * **Gol vs xG:** Andrey Santos dan Xavi Simons menunjukkan tingkat konversi Gol/xG yang sangat tinggi (1.96 - newplot (24).png), menandakan *finishing* klinis yang melebihi ekspektasi peluang.
    * **Assist vs xAG:** Benjamin Šeško (2.38) dan Jude Bellingham (2.05) memiliki tingkat konversi Assist/xAG yang sangat tinggi (newplot (22).png), menunjukkan kemampuan untuk memberikan umpan yang menghasilkan *assist* lebih dari yang diperkirakan.

**Kesimpulan Umum:**

Ekosistem pengembangan pemain muda di Top 5 Liga sangat bervariasi. **Ligue 1** menonjol dalam kuantitas dan keberanian memberikan kesempatan, menghasilkan banyak talenta ofensif. **La Liga** adalah lingkungan terbaik untuk pemain muda yang mencari peran *starter* dan menjadi *playmaker* inti, dengan Pedri sebagai contoh utama. **Premier League** menawarkan menit bermain yang substansial di lingkungan yang kompetitif. **Bundesliga** menunjukkan efisiensi tinggi dari para pemain mudanya dalam produksi gol dan *assist*. Sementara **Serie A** mungkin lebih menantang dalam hal menit bermain, ia tetap menghasilkan pemain muda dengan efektivitas individu yang tinggi.

Secara kolektif, pemain muda sudah memberikan dampak signifikan di lini serang dan menjadi pilar penting di berbagai posisi. Kemampuan mereka untuk tidak hanya bermain, tetapi juga secara konsisten menjadi *starter*, mencetak gol, membuat *assist*, dan memajukan permainan (melalui umpan progresif) adalah bukti nyata potensi besar yang akan membentuk masa depan sepak bola.

# Clustering & Clastification

In [88]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Pilih fitur peran
features = [
    'Gls', 'G-PK', 'xG', 'Ast', 'xAG', 'PrgP', 'CrdY', 'Min'
]

# Ambil data pemain muda
young_players = df_cleaned[df_cleaned['Age'] <= age_threshold].copy()

# Filter dan drop NA
X = young_players[features].fillna(0)

# Standardisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42)
young_players['RoleCluster'] = kmeans.fit_predict(X_scaled)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)
young_players['PC1'] = pca_result[:, 0]
young_players['PC2'] = pca_result[:, 1]

fig = px.scatter(
    young_players,
    x='PC1', y='PC2',
    color='RoleCluster',
    hover_data=['Player', 'Squad', 'Age'],
    title='Klastering Peran Pemain Muda Berdasarkan Statistik Performa'
)
fig.show()

In [89]:
cluster_summary = young_players.groupby('RoleCluster')[features].mean().round(2)
display(cluster_summary)

# Misalnya setelah melihat cluster_summary hasilnya:
# Cluster 0 → Finisher
# Cluster 1 → Defender
# Cluster 2 → Playmaker

role_map = {
    0: "Finisher",
    1: "Defender",
    2: "Playmaker"
}

young_players['Role'] = young_players['RoleCluster'].map(role_map)

fig = px.scatter(
    young_players,
    x='PC1', y='PC2',
    color='Role',
    hover_data=['Player', 'Squad', 'Age', 'Gls', 'Ast'],
    title='Klasifikasi Peran Pemain Muda Berdasarkan Statistik Performa'
)
fig.show()


,Gls,G-PK,xG,Ast,xAG,PrgP,CrdY,Min
RoleCluster,,,,,,,,
0,1.75,1.71,1.82,1.86,1.74,71.97,4.18,1753.38
1,0.26,0.26,0.31,0.17,0.22,8.57,0.63,276.67
2,8.42,8.00,7.36,5.15,5.08,99.52,3.97,2203.97


In [54]:
young_players['Role'].value_counts()

Role
Defender     509
Finisher     150
Playmaker     33
Name: count, dtype: int64

In [55]:
role_count = young_players['Role'].value_counts().reset_index()
role_count.columns = ['Role', 'Jumlah Pemain']

fig = px.bar(
    role_count,
    x='Role',
    y='Jumlah Pemain',
    color='Role',
    title='Distribusi Peran Pemain Muda',
    text='Jumlah Pemain',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition='outside')
fig.update_layout(title_x=0.5)
fig.show()


In [56]:
avg_stats = young_players.groupby('Role')[['Gls', 'Ast', 'xG', 'xAG']].mean().round(2).reset_index()

fig = px.bar(
    avg_stats.melt(id_vars='Role'),
    x='Role',
    y='value',
    color='variable',
    barmode='group',
    title='Rata-rata Statistik Kunci per Peran',
    labels={'value': 'Rata-rata', 'variable': 'Statistik'},
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig.update_layout(title_x=0.5)
fig.show()
